In [54]:
# _*_ coding: utf-8 _*_
# @Time: 2021/11/15 17:27 
# @Author: yuyongsheng
# @Software: PyCharm
# @Description: 上海六院甲氨蝶呤：2. 从数据库中提取数据

# 原始数据预处理

## 导入程序包

In [55]:
# 导入程序包
import pymysql as MySQLDB
import pandas as pd
import numpy as np

import re
import sys
import os
project_path = os.getcwd()

## 导入预定义函数

In [56]:
# 字符串转换为时间格式
import datetime
def str_to_datetime(x):
    try:
        a = datetime.datetime.strptime(x, "%d/%m/%Y %H:%M:%S")
        return a
    except:
        return np.NaN
        

In [57]:
# 过滤文字
import numpy as np
def filter_string(df,feature):
    # 过滤文字!!!!!!!!!!!!!!!!!!!!!!!!!!!
    df=df[df[feature].str.contains('\d')]
    return df

In [58]:
# 过滤异常大值
def filter_exception_value(df,feature,median_value=1):
    # 过滤异常大值!!!!!!!!!!!!!!!!!!!!!!!!!!
    if not median_value:
        median_value=df[feature].median()
    df[feature]=df[feature].apply(lambda x: x if abs(float(x)) < (100 * abs(median_value)) else np.nan)
    df=df[df[feature].notnull()]
    return df

In [59]:
# 使用随机森林对缺失值进行插补
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
def missing_value_interpolation(df,missing_list=[]):
    df = df.reset_index(drop=True)
    # 提取存在缺失值的列名
    if not missing_list:
        for i in df.columns:
            if df[i].isnull().sum() > 0:
                missing_list.append(i)
    missing_list_copy = missing_list.copy()
    # 用该列未缺失的值训练随机森林，然后用训练好的rf预测缺失值
    for i in range(len(missing_list)):
        name=missing_list[0]
        df_missing = df[missing_list_copy]
        # 将其他列的缺失值用0表示。
        missing_list.remove(name)
        for j in missing_list:
            df_missing[j]=df_missing[j].astype('str').apply(lambda x: 0 if x=='nan' else x)
        df_missing_is = df_missing[df_missing[name].isnull()]
        df_missing_not = df_missing[df_missing[name].notnull()]
        y = df_missing_not[name]
        x = df_missing_not.drop([name],axis=1)
        # 列出参数列表
        tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
        # 进行参数的搜索组合
        grid = GridSearchCV(RandomForestRegressor(),param_grid=tree_grid_parameter,cv=3)
        #rfr=RandomForestRegressor(random_state=0,n_estimators=100,n_jobs=-1)
        #根据已有数据去拟合随机森林模型
        grid.fit(x, y)
        rfr = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'])
        rfr.fit(x, y)
        #预测缺失值
        predict = rfr.predict(df_missing_is.drop([name],axis=1))
        #填补缺失值
        df.loc[df[name].isnull(),name] = predict
    return df

## 连接数据库

In [394]:
# 连接数据库
conn = MySQLDB.connect(host='192.168.0.37', port=3306, user='root', password='111111', db='inp', charset='UTF8')
# conn = MySQLDB.connect(host='localhost', port=3306, user='root', password='123456', db='上海六院', charset='UTF8')
cursor = conn.cursor()
cursor.execute("Select version()")
for i in cursor:
    print(i)

('8.0.22',)


## 读取inp_record住院记录

In [68]:
# 从inp_record表读入数据
try:
    sql = 'select * from inp_record_zs;'
    df_inp_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [69]:
print(df_inp_record.shape)
print(df_inp_record['patient_id'].nunique())

(35897, 8)
12023


### 提取人口信息学数据

In [70]:
# 提取人口信息学数据
df_popu=df_inp_record[['patient_id','gender','age']]

In [71]:
print(df_popu.shape)

(35897, 3)


In [72]:
# 删除重复数据
df_popu = df_popu.drop_duplicates(subset=['patient_id','gender'],keep='first')

In [73]:
print(df_popu.shape)
print(df_popu['patient_id'].nunique())

(12025, 3)
12023


In [74]:
# 保存人口学特征
writer=pd.ExcelWriter(project_path+'/data/pre_processed_raw_data/df_人口学特征.xlsx')
df_popu.to_excel(writer)
writer.save()

## 读取medical_record数据库表

In [395]:
# 从medical_record表读入数据
try:
    sql = 'select * from medical_record_zs;'
    df_medical_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [396]:
print(df_medical_record.shape)
print(df_medical_record['patient_id'].nunique())

(69808, 5)
11520


In [397]:
# 删除既往史为空的记录
df_medical_record=df_medical_record[df_medical_record['record_content'].notnull()]
df_medical_record=df_medical_record.reset_index(drop=True)

In [398]:
print(df_medical_record.shape)
print(df_medical_record['patient_id'].nunique())

(69359, 5)
11505


In [41]:
writer=pd.ExcelWriter(project_path+'/data/pre_processed_raw_data/df_既往史.xlsx')
df_medical_record.to_excel(writer)
writer.save()

## 读取doctor_order数据库表

In [38]:
# 从df_doctor_order数据库读入数据
try:
    sql = 'select * from doctor_order_zs1;'
    df_doctor_order = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [39]:
print(df_doctor_order.shape)
print(df_doctor_order['patient_id'].nunique())

(718705, 16)
11824


In [40]:
# 提取长期医嘱
df_doctor_order=df_doctor_order[df_doctor_order['long_d_order']=='长期医嘱']

In [41]:
print(df_doctor_order.shape)
print(df_doctor_order['patient_id'].nunique())

(285723, 16)
6258


In [42]:
# 删除end_datetime为空的数据
df_doctor_order=df_doctor_order[df_doctor_order['end_datetime'].notnull()]

In [43]:
print(df_doctor_order.shape)
print(df_doctor_order['patient_id'].nunique())

(283170, 16)
6224


In [44]:
# 删除重复数据
df_doctor_order=df_doctor_order.drop_duplicates(subset=['patient_id','case_no','drug_name','dosage','frequency','start_datetime','end_datetime'],keep='first')
df_doctor_order=df_doctor_order.reset_index(drop=True)

In [45]:
print(df_doctor_order.shape)
print(df_doctor_order['patient_id'].nunique())
print(df_doctor_order['case_no'].nunique())

(282961, 16)
6224
22824


In [46]:
# 提取doctor_order里面的有效字段
df_doctor_order=df_doctor_order[['patient_id','case_no','long_d_order','drug_name','amount','drug_spec','dosage','frequency','medication_way','start_datetime','end_datetime']]

In [47]:
df_doctor_order

,patient_id,case_no,long_d_order,drug_name,amount,drug_spec,dosage,frequency,medication_way,start_datetime,end_datetime
0,287873,355679,长期医嘱,(甲)0.9%氯化钠注射液,1.0,250ml/袋,1.000,BID,静滴,2010-05-27 09:41:00,2010-05-29 09:41:00
1,287873,355679,长期医嘱,(甲)注射用头孢呋辛钠（丽扶欣）,2.0,1g/瓶,2.000,BID,静滴,2010-05-27 09:41:00,2010-05-29 09:41:00
2,287873,355679,长期医嘱,(甲)0.9%氯化钠注射液,1.0,250ml/袋,1.000,QD,静滴,2010-05-27 09:41:00,2010-05-29 09:41:00
3,287873,355679,长期医嘱,(甲)甘草酸二铵注射液（甘利欣）,3.0,50mg*5支/合,3.000,QD,静滴,2010-05-27 09:41:00,2010-05-29 09:41:00
4,287873,355679,长期医嘱,(乙10%)注射用还原型谷胱甘肽（阿拓莫兰),3.0,0.6g/支,3.000,QD,静滴,2010-05-27 09:41:00,2010-05-29 09:41:00
...,...,...,...,...,...,...,...,...,...,...,...
282956,869255,1231198,长期医嘱,※(乙20%)盐酸帕洛诺司琼注射液（欧赛）,1.0,5ml:0.25mg/支,0.250,QD,静滴,2019-05-03 09:09:08,2019-05-04 09:09:00
282957,869255,1231198,长期医嘱,(甲)地塞米松磷酸钠注射液,1.0,5mg*1ml*10支/合,5.000,QD,静滴,2019-05-03 09:09:08,2019-05-04 09:09:00
282958,869255,1231198,长期医嘱,(甲)0.9%氯化钠注射液,1.0,100ml/袋,100.000,QD,静滴,2019-05-03 09:09:08,2019-05-04 09:09:00
282959,869255,1231198,长期医嘱,(乙)注射用异环磷酰胺（和乐生）,3.0,1g/瓶,3.000,QD,静滴,2019-05-03 09:09:08,2019-05-05 09:09:00


In [52]:
# 保存doctor_order用药记录
writer=pd.ExcelWriter(project_path+'/data/pre_processed_raw_data/df_doctor_order.xlsx')
df_doctor_order.to_excel(writer)
writer.save()

## 读取test_result数据库表

In [6]:
# 连接数据库
conn = MySQLDB.connect(host='192.168.0.37', port=3306, user='root', password='111111', db='inp', charset='UTF8')
# conn = MySQLDB.connect(host='localhost', port=3306, user='root', password='123456', db='上海六院', charset='UTF8')
cursor = conn.cursor()
cursor.execute("Select version()")
for i in cursor:
    print(i)

('8.0.22',)


In [7]:
# 从test_result数据库读入数据
try:
    sql = 'select * from test_result_zs;'
    df_test_result = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [8]:
print(df_test_result.shape)
print(df_test_result['patient_id'].nunique())

(4015506, 11)
10549


In [9]:
# 删除test_result为空的数据
df_test_result=df_test_result[df_test_result['test_result'].notnull()]
df_test_result=df_test_result.reset_index(drop=True)

In [10]:
print(df_test_result.shape)

(4015501, 11)


In [11]:
# 删除test_result重复数据
df_test_result=df_test_result.drop_duplicates(subset=['patient_id','case_no','test_date','project_code','project_name','test_result','is_normal','refer_scope'],keep='first')
df_test_result=df_test_result.reset_index(drop=True)

In [12]:
print(df_test_result.shape)

(4015097, 11)


In [13]:
# 删除<>号
df_test_result['test_result']=df_test_result['test_result'].astype('str').apply(lambda x:x.replace('<',''))
df_test_result['test_result']=df_test_result['test_result'].astype('str').apply(lambda x:x.replace('>',''))

In [14]:
print(df_test_result.shape)
print(df_test_result['patient_id'].nunique())

(4015097, 11)
10549


# 纳排

## 提取用药数据

### 提取甲氨蝶呤的用药数据

In [48]:
# 提取甲氨蝶呤的用药数据
# 注意药物缩写和药物音译名，wtf
df_MTX = df_doctor_order[df_doctor_order['drug_name'].str.contains('甲氨')]

In [49]:
df_MTX['patient_id'].nunique()

721

In [50]:
# 人工删除剂量小于5的数据
df_MTX=df_MTX[df_MTX['dosage'].astype('float')>=5]

In [51]:
# 同一case_no，取第一次用药记录
df_MTX = df_MTX.sort_values(by=['patient_id','case_no','start_datetime'])
df_MTX = df_MTX.drop_duplicates(subset=['patient_id','case_no'],keep='first')

In [52]:
# # 取第一次入院
# df_MTX = df_MTX.sort_values(by=['patient_id','start_datetime'],ascending=[True,True])
# df_MTX = df_MTX.drop_duplicates(['patient_id'],keep='first')

In [61]:
# 删除异常数据
df_MTX=filter_exception_value(df_MTX,'dosage')
df_MTX = df_MTX.reset_index(drop=True)

In [512]:
print(df_MTX.shape)
print(df_MTX['patient_id'].nunique())
print(df_MTX['case_no'].nunique())

(1321, 11)
718
1321


In [64]:
# 保存甲氨蝶呤用药数据
writer=pd.ExcelWriter(project_path+'/data/processed_data/用药/df_MTX.xlsx')
df_MTX.to_excel(writer)
writer.save()

### 提取阿霉素/多柔比星的用药数据

In [65]:
# 提取阿霉素的用药数据，不能包含多柔比星脂质体，因为方案要区分
df_ADM = df_doctor_order[df_doctor_order['drug_name'].str.contains('注射用盐酸多柔比星')]
df_ADM = df_ADM.reset_index(drop=True)

In [66]:
print(df_ADM.shape)
print(df_ADM['patient_id'].nunique())
print(df_ADM['case_no'].nunique())

(1950, 11)
919
1819


In [75]:
df_ADM=pd.merge(df_ADM,df_popu,on=['patient_id'],how='left')

In [76]:
# 保存阿霉素用药数据
writer=pd.ExcelWriter(project_path+'/data/processed_data/用药/df_ADM.xlsx')
df_ADM.to_excel(writer)
writer.save()

### 提取异环磷酰胺的用药数据

In [77]:
# 提取异环磷酰胺的用药数据
df_IFO = df_doctor_order[df_doctor_order['drug_name'].str.contains('异环磷酰胺|IFO')]
df_IFO = df_IFO.reset_index(drop=True)

In [78]:
print(df_IFO.shape)
print(df_IFO['patient_id'].nunique())
print(df_IFO['case_no'].nunique())

(5404, 11)
1691
4214


In [79]:
df_IFO=pd.merge(df_IFO,df_popu,on=['patient_id'],how='left')

In [80]:
writer=pd.ExcelWriter(project_path+'/data/processed_data/用药/df_IFO.xlsx')
df_IFO.to_excel(writer)
writer.save()

### 提取顺铂的用药数据

In [81]:
# 提取顺铂的用药数据
df_CDP = df_doctor_order[df_doctor_order['drug_name'].str.contains('顺铂|CDP')]
df_CDP = df_CDP.reset_index(drop=True)

In [82]:
print(df_CDP.shape)
print(df_CDP['patient_id'].nunique())
print(df_CDP['case_no'].nunique())

(5942, 11)
2157
5402


In [83]:
df_CDP=pd.merge(df_CDP,df_popu,on=['patient_id'],how='left')

In [84]:
# 保存顺铂的用药数据
writer=pd.ExcelWriter(project_path+'/data/processed_data/用药/df_CDP.xlsx')
df_CDP.to_excel(writer)
writer.save()

## 提取不良反应检测

### 提取肾毒性相关检测

#### 提取肾小球滤过率降低

In [89]:
# 提取肾小球滤过率
df_test_GFR=df_test_result[df_test_result['project_code'].str.contains('GFR')]
df_test_GFR=df_test_GFR.reset_index(drop=True)

In [90]:
print(df_test_GFR.shape)
print(df_test_GFR['patient_id'].nunique())

(5740, 11)
3347


In [91]:
# 保存肾小球滤过率检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/肾毒性/df_temp_肾小球滤过率.xlsx')
df_test_GFR.to_excel(writer)
writer.save()

In [94]:
# # 方案（一）：根据数据库is_normal字段，提取小球滤过率降低
# df_test_GFR_low=df_test_GFR[df_test_GFR['is_normal']=='L']
# df_test_GFR_low=df_test_GFR_low.reset_index(drop=True)

In [97]:
# 方案（二）：根据本善提供不良反应指标，提取肾小球滤过率（GFR）<125ml/min、肾小球过滤率（eGFR）<60ml/min/1.73m^2
df_test_GFR_low=df_test_GFR[(~ df_test_GFR['project_code'].str.contains('eGFR')) & (df_test_GFR['test_result'].astype('float')<125)]

In [99]:
df_test_eGFR_low=df_test_GFR[(df_test_GFR['project_code'].str.contains('eGFR')) & (df_test_GFR['test_result'].astype('float')<60)]

In [101]:
df_test_GFR_low =pd.concat([df_test_GFR_low,df_test_eGFR_low],axis=0)
df_test_GFR_low = df_test_GFR_low.reset_index(drop=True)

In [102]:
print(df_test_GFR_low.shape)
print(df_test_GFR_low['patient_id'].nunique())
print(df_test_GFR_low['case_no'].nunique())

(1123, 11)
629
1091


In [103]:
# 保存肾小球滤过率（GFR）<125ml、GFR）<125ml/min、（eGFR）<60ml的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/肾毒性/df_temp_肾小球滤过率降低.xlsx')
df_test_GFR_low.to_excel(writer)
writer.save()

#### 提取血清尿素氮(BUN)升高

In [104]:
# 提取血清尿素氮检测结果
df_test_BUN=df_test_result[df_test_result['project_code'].str.contains('BUN')]
df_test_BUN=df_test_BUN.reset_index(drop=True)

In [105]:
df_test_BUN

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
0,11,665849,918995,2016-08-06,血,VBUN-J,尿素(干式),5.5,mmol/L,None,2.5～7.1
1,94,432942,570519,2013-01-04,血,BUN,尿素,4.2,mmol/L,None,2.5～6.4
2,116,665849,918995,2016-08-04,血,BUN,尿素,4.6,mmol/L,None,2.5～6.4
3,199,432942,570519,2013-01-07,血,VBUN-J,尿素(干式),4.6,mmol/L,None,2.5～7.1
4,348,712410,1176403,2018-11-16,血清,BUN,尿素,5.7,mmol/L,None,2.5～6.4
...,...,...,...,...,...,...,...,...,...,...,...
54212,4015264,945291,1345066,2020-05-20,血浆,VBUN-J,尿素(干式),2.9,mmol/L,None,2.5～7.1
54213,4015311,945291,1345066,2020-05-25,血浆,VBUN-J,尿素(干式),2.8,mmol/L,None,2.5～7.1
54214,4015332,959410,1365759,2020-07-29,血浆,VBUN-J,尿素(干式),3.7,mmol/L,None,2.5～7.1
54215,4015406,610802,835370,2015-10-16,血,BUN,尿素,4.8,mmol/L,None,2.5～6.4


In [106]:
# 保存血清尿素氮检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/肾毒性/df_temp_血清尿素氮.xlsx')
df_test_BUN.to_excel(writer)
writer.save()

In [107]:
# 方案（一）：根据数据库is_normal字段，提取血清尿素氮升高检测结果
df_test_BUN_high=df_test_BUN[df_test_BUN['is_normal']=='H']
df_test_BUN_high=df_test_BUN_high.reset_index(drop=True)

In [108]:
df_test_BUN_high

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
0,1489,494789,690347,2014-05-05,血,VBUN-J,尿素(干式),8.0,mmol/L,H,2.5～7.1
1,5201,792835,1119411,2018-05-29,血清,BUN,尿素,7.2,mmol/L,H,2.5～6.4
2,6795,432641,570338,2013-01-04,血,VBUN-J,尿素(干式),15.7,mmol/L,H,2.5～7.1
3,7305,549276,740971,2014-11-05,血,VBUN-J,尿素(干式),8.0,mmol/L,H,2.5～7.1
4,7399,432641,570338,2013-01-02,血,VBUN-J,尿素(干式),15.4,mmol/L,H,2.5～7.1
...,...,...,...,...,...,...,...,...,...,...,...
3768,4010175,1019017,1453799,2021-04-28,血清,BUN,尿素,6.6,mmol/L,H,2.5～6.4
3769,4012198,486851,736034,2014-10-17,血,BUN,尿素,7.3,mmol/L,H,2.5～6.4
3770,4012974,724405,1032087,2017-08-21,血清,BUN,尿素,6.8,mmol/L,H,2.5～6.4
3771,4013055,436289,591635,2013-04-10,血,VBUN-J,尿素(干式),8.3,mmol/L,H,2.5～7.1


In [109]:
print(df_test_BUN_high.shape)
print(df_test_BUN_high['patient_id'].nunique())
print(df_test_BUN_high['case_no'].nunique())

(3773, 11)
1889
3060


In [110]:
# 保存血清尿素氮升高的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/肾毒性/df_temp_血清尿素氮升高.xlsx')
df_test_BUN_high.to_excel(writer)
writer.save()

In [111]:
# # 方案（二）：根据本善提供不良反应指标，提取血清尿素氮升高检测结果,尿素氮（BUN）>7.5mmol/L(21mg/dl)
df_test_BUN_high=df_test_BUN[df_test_BUN['test_result'].astype('float')>7.5]
df_test_BUN_high=df_test_BUN_high.reset_index(drop=True)

In [112]:
df_test_BUN_high

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
0,1489,494789,690347,2014-05-05,血,VBUN-J,尿素(干式),8.0,mmol/L,H,2.5～7.1
1,6795,432641,570338,2013-01-04,血,VBUN-J,尿素(干式),15.7,mmol/L,H,2.5～7.1
2,7305,549276,740971,2014-11-05,血,VBUN-J,尿素(干式),8.0,mmol/L,H,2.5～7.1
3,7399,432641,570338,2013-01-02,血,VBUN-J,尿素(干式),15.4,mmol/L,H,2.5～7.1
4,8327,432641,570338,2013-01-10,血,VBUN-J,尿素(干式),7.6,mmol/L,H,2.5～7.1
...,...,...,...,...,...,...,...,...,...,...,...
2066,3992138,1010210,1449053,2021-04-15,血浆,VBUN-J,尿素(干式),7.7,mmol/L,H,2.5～7.1
2067,3996834,521845,1431081,2021-02-23,血清,BUN,尿素,12.5,mmol/L,H,2.5～6.4
2068,3998770,1004862,1433119,2021-03-01,血浆,VBUN-J,尿素(干式),8.6,mmol/L,H,2.5～7.1
2069,4000850,1024829,1462279,2021-05-27,血清,BUN,尿素,7.8,mmol/L,H,2.5～6.4


In [113]:
print(df_test_BUN_high.shape)
print(df_test_BUN_high['patient_id'].nunique())
print(df_test_BUN_high['case_no'].nunique())

(2071, 11)
1076
1642


In [114]:
# 保存血清尿素氮升高的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/肾毒性/df_temp_血清尿素氮升高_本善标准.xlsx')
df_test_BUN_high.to_excel(writer)
writer.save()

#### 提取血肌酐(SCr)升高

In [115]:
# 提取血肌酐检测结果
df_test_SCr=df_test_result[df_test_result['project_code'].str.contains('CREA') & (df_test_result['sample'].str.contains('血'))]
df_test_SCr=filter_string(df_test_SCr,'test_result')
df_test_SCr=df_test_SCr.reset_index(drop=True)

In [116]:
print(df_test_SCr.shape)
print(df_test_SCr['patient_id'].nunique())
print(df_test_SCr['case_no'].nunique())

(54332, 11)
10344
27354


In [117]:
df_test_SCr

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
0,9,665849,918995,2016-08-06,血,VCREA-J,肌酐(干式),73,μmol/L,None,62～133
1,99,432942,570519,2013-01-04,血,CREA,肌酐,57,μmol/L,None,53～115
2,111,665849,918995,2016-08-04,血,CREA,肌酐,78,μmol/L,None,53～115
3,202,432942,570519,2013-01-07,血,VCREA-J,肌酐(干式),38,μmol/L,L,62～133
4,341,712410,1176403,2018-11-16,血清,CREA,肌酐,60,μmol/L,None,53～115
...,...,...,...,...,...,...,...,...,...,...,...
54327,4015261,945291,1345066,2020-05-20,血浆,VCREA-J,肌酐(干式),61,μmol/L,None,58～110
54328,4015308,945291,1345066,2020-05-25,血浆,VCREA-J,肌酐(干式),50,μmol/L,L,58～110
54329,4015329,959410,1365759,2020-07-29,血浆,VCREA-J,肌酐(干式),54,μmol/L,L,58～110
54330,4015401,610802,835370,2015-10-16,血,CREA,肌酐,53,μmol/L,None,53～115


In [118]:
# 保存血肌酐检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/肾毒性/df_temp_血肌酐.xlsx')
df_test_SCr.to_excel(writer)
writer.save()

In [119]:
# 方案（一）：根据数据库_normal字段，提取血肌酐升高检测结果
df_test_SCr_high=df_test_SCr[df_test_SCr['is_normal']=='H']
df_test_SCr_high=df_test_SCr_high.reset_index(drop=True)

In [120]:
df_test_SCr_high

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
0,30868,793308,1115695,2018-05-18,血清,VCREA-J,肌酐(干式),97,μmol/L,H,46～92
1,35903,731771,1020194,2017-07-13,血清,CREA,肌酐,123,μmol/L,H,53～115
2,46333,559758,817374,2015-08-16,血,VCREA-J,肌酐(干式),138,μmol/L,H,62～133
3,56527,863056,1280447,2019-09-20,血清,VCREA-J,肌酐(干式),93,μmol/L,H,46～92
4,59322,863056,1280447,2019-09-22,血清,VCREA-J,肌酐(干式),105,μmol/L,H,46～92
...,...,...,...,...,...,...,...,...,...,...,...
336,3954365,996583,1446396,2021-04-12,血浆,VCREA-J,肌酐(干式),228,μmol/L,H,46～92
337,3954385,996583,1446396,2021-04-13,血浆,VCREA-J,肌酐(干式),223,μmol/L,H,46～92
338,3982961,920615,1307917,2019-12-06,血清,CREA,肌酐,124,μmol/L,H,53～115
339,4003223,664265,1121297,2018-06-04,血清,VCREA-J,肌酐(干式),94,μmol/L,H,46～92


In [121]:
print(df_test_SCr_high.shape)
print(df_test_SCr_high['patient_id'].nunique())
print(df_test_SCr_high['case_no'].nunique())

(341, 11)
165
223


In [122]:
# 保存血肌酐升高的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/肾毒性/df_temp_血肌酐升高.xlsx')
df_test_SCr_high.to_excel(writer)
writer.save()

In [123]:
# 方案（二）：根据本善不良反应指标，提取血肌酐升高检测结果。
#（男：>133umol/L（1.2mg/dl）、女：>97umol/L（1.1mg/dl升）、小儿：>69.7umol/L）
# 加入人口学特征
df_test_SCr_age=pd.merge(df_test_SCr,df_popu,on=['patient_id'],how='left')

In [124]:
print(df_test_SCr_age.shape)
print(df_test_SCr_age['patient_id'].nunique())

(54332, 13)
10344


In [125]:
# 统计18岁以下的
df_test_SCr_young=df_test_SCr_age[df_test_SCr_age['age'].astype('int')<18]

In [126]:
print(df_test_SCr_young.shape)
print(df_test_SCr_young['patient_id'].nunique())

(15456, 13)
2329


In [127]:
# 小儿：>69.7umol/L
df_test_SCr_young_high=df_test_SCr_young[df_test_SCr_young['test_result'].astype('float')>69.7]
df_test_SCr_young_high=df_test_SCr_young_high.reset_index(drop=True)

In [128]:
print(df_test_SCr_young_high.shape)
print(df_test_SCr_young_high['patient_id'].nunique())

(476, 13)
210


In [129]:
print(df_test_SCr_young_high.to_string())

          id patient_id  case_no  test_date sample          project_code project_name test_result          result_unit is_normal refer_scope gender  age
0       4463     865292  1225280 2019-04-15     血清  CREA                           肌酐          73  μmol/L                   None      53～115    男     16
1      16457     727362  1013510 2017-06-22     血清  CREA                           肌酐          82  μmol/L                   None      53～115    男     17
2      28512     610009  1219466 2019-03-30     血清  VCREA-J                    肌酐(干式)          70  μmol/L                   None      58～110    男     15
3      30892     373405   478376 2011-12-08      血  CREA                           肌酐          74  μmol/L                   None      53～115    男     17
4      41481     938708  1335324 2020-04-14     血清  CREA                           肌酐          71  μmol/L                   None      53～115    男     16
5      42839     311638   401539 2010-12-22      血  CREA                          

In [130]:
# 统计18岁以上的
df_test_SCr_adult=df_test_SCr_age[df_test_SCr_age['age'].astype('int')>=18]

In [131]:
print(df_test_SCr_adult.shape)
print(df_test_SCr_adult['patient_id'].nunique())

(38876, 13)
8015


In [132]:
df_test_SCr_adult

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope,gender,age
0,9,665849,918995,2016-08-06,血,VCREA-J,肌酐(干式),73,μmol/L,None,62～133,女,26
1,99,432942,570519,2013-01-04,血,CREA,肌酐,57,μmol/L,None,53～115,女,37
2,111,665849,918995,2016-08-04,血,CREA,肌酐,78,μmol/L,None,53～115,女,26
3,202,432942,570519,2013-01-07,血,VCREA-J,肌酐(干式),38,μmol/L,L,62～133,女,37
4,341,712410,1176403,2018-11-16,血清,CREA,肌酐,60,μmol/L,None,53～115,女,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54327,4015261,945291,1345066,2020-05-20,血浆,VCREA-J,肌酐(干式),61,μmol/L,None,58～110,男,27
54328,4015308,945291,1345066,2020-05-25,血浆,VCREA-J,肌酐(干式),50,μmol/L,L,58～110,男,27
54329,4015329,959410,1365759,2020-07-29,血浆,VCREA-J,肌酐(干式),54,μmol/L,L,58～110,男,18
54330,4015401,610802,835370,2015-10-16,血,CREA,肌酐,53,μmol/L,None,53～115,女,49


In [133]:
# （男：>133umol/L（1.2mg/dl）
df_test_SCr_male=df_test_SCr_adult[df_test_SCr_adult['gender'].str.contains('男')]
df_test_SCr_male_high=df_test_SCr_male[df_test_SCr_male['test_result'].astype('float')>133]
df_test_SCr_male_high=df_test_SCr_male_high.reset_index(drop=True)

In [134]:
print(df_test_SCr_male_high.shape)
print(df_test_SCr_male_high['patient_id'].nunique())

(89, 13)
42


In [135]:
# 女：>97umol/L（1.1mg/dl升）
df_test_SCr_female=df_test_SCr_adult[df_test_SCr_adult['gender'].str.contains('女')]
df_test_SCr_female_high=df_test_SCr_female[df_test_SCr_female['test_result'].astype('float')>97]
df_test_SCr_female_high=df_test_SCr_female_high.reset_index(drop=True)

In [136]:
df_test_SCr_female_high

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope,gender,age
0,24357,551551,837898,2015-11-02,血,VCREA-J,肌酐(干式),124,μmol/L,None,62～133,女,43
1,25073,551551,837898,2015-11-04,血,VCREA-J,肌酐(干式),124,μmol/L,None,62～133,女,43
2,51578,863056,1280447,2019-09-19,血清,CREA,肌酐,110,μmol/L,None,53～115,女,73
3,59322,863056,1280447,2019-09-22,血清,VCREA-J,肌酐(干式),105,μmol/L,H,46～92,女,73
4,80204,551551,823118,2015-08-29,血,CREA,肌酐,107,μmol/L,None,53～115,女,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,3930324,664265,1058909,2017-11-17,血清,CREA,肌酐,100,μmol/L,None,53～115,女,67
204,3950634,996583,1446396,2021-04-10,血浆,VCREA-J,肌酐(干式),379,μmol/L,H,46～92,女,72
205,3951653,996583,1446396,2021-04-11,血浆,VCREA-J,肌酐(干式),307,μmol/L,H,46～92,女,72
206,3954365,996583,1446396,2021-04-12,血浆,VCREA-J,肌酐(干式),228,μmol/L,H,46～92,女,72


In [137]:
print(df_test_SCr_female_high.shape)
print(df_test_SCr_female_high['patient_id'].nunique())

(208, 13)
86


In [138]:
df_test_SCr_adult_high=pd.concat([df_test_SCr_male_high,df_test_SCr_female_high],axis=0)
df_test_SCr_high=pd.concat([df_test_SCr_adult_high,df_test_SCr_young_high],axis=0)
df_test_SCr_high=df_test_SCr_high.reset_index(drop=True)

In [139]:
print(df_test_SCr_high.shape)
print(df_test_SCr_high['patient_id'].nunique())

(773, 13)
338


In [140]:
# 保存血肌酐升高的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/肾毒性/df_temp_血肌酐升高_本善标准.xlsx')
df_test_SCr_high.to_excel(writer)
writer.save()

#### 合并肾小球滤过率降低、血清尿素氮和肌酐升高检测记录

In [141]:
# 合并心脏毒性所有检测
df_test_kidney_toxicity=pd.concat([df_test_GFR,df_test_BUN],axis=0)
df_test_kidney_toxicity=pd.concat([df_test_kidney_toxicity,df_test_SCr],axis=0)

In [142]:
# 排序
df_test_kidney_toxicity=df_test_kidney_toxicity.sort_values(['patient_id','test_date'])
df_test_kidney_toxicity=df_test_kidney_toxicity.reset_index(drop=True)

In [143]:
print(df_test_kidney_toxicity.shape)
print(df_test_kidney_toxicity['patient_id'].nunique())
print(df_test_kidney_toxicity['case_no'].nunique())

(114289, 11)
10344
27354


In [144]:
# 保存骨髓抑制的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_2.1.1_肾毒性所有检测.xlsx')
df_test_kidney_toxicity.to_excel(writer)
writer.save()

In [145]:
df_test_kidney_toxicity_low=pd.concat([df_test_GFR_low,df_test_BUN_high],axis=0)
df_test_kidney_toxicity_low=pd.concat([df_test_kidney_toxicity_low,df_test_SCr_high],axis=0)
df_test_kidney_toxicity_low=df_test_kidney_toxicity_low.reset_index(drop=True)

In [146]:
print(df_test_kidney_toxicity_low.shape)
print(df_test_kidney_toxicity_low['patient_id'].nunique())

(3967, 13)
1743


In [147]:
# 保存肾毒性检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_2.1.2_肾毒性指标降低的所有检测.xlsx')
df_test_kidney_toxicity_low.to_excel(writer)
writer.save()

In [148]:
# 提取甲氨蝶呤服药患者的肾毒性记录
df_test_kidney_toxicity_MTX=df_test_kidney_toxicity[df_test_kidney_toxicity['patient_id'].isin(np.unique(df_MTX['patient_id']))]

In [149]:
print(df_test_kidney_toxicity_MTX.shape)
print(df_test_kidney_toxicity_MTX['patient_id'].nunique())

(30376, 11)
718


In [150]:
# 保存甲氨蝶呤服药患者的肾毒性记录
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_2.1_肾毒性_甲氨蝶呤.xlsx')
df_test_kidney_toxicity_MTX.to_excel(writer)
writer.save()

### 提取心脏毒性相关检测

#### 提取心肌酶谱升高的检测结果

In [151]:
# 心肌酶谱 ( CK、CK-MB、LDH 、AST、α -HBDH) 。
df_test_xjmp=df_test_result[(df_test_result['project_code'].str.contains('CK|CK-MB|LDH|AST|HBDH'))
                           & (df_test_result['sample'].str.contains('血'))]
# 删除文字检测结果
df_test_xjmp=filter_string(df_test_xjmp,'test_result')
df_test_xjmp=df_test_xjmp.reset_index(drop=True)

In [152]:
# 保存心肌酶谱检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/心脏毒性/df_temp_心肌酶谱.xlsx')
df_test_xjmp.to_excel(writer)
writer.save()

In [153]:
# 方案（一）：按照is_normal，提取心肌酶谱升高
df_test_xjmp_high=df_test_xjmp[df_test_xjmp['is_normal']=='H']
df_test_xjmp_high=df_test_xjmp_high.reset_index(drop=True)

In [331]:
df_test_xjmp_high

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
0,1228,792697,1144585,2018-08-11,血清,CK,肌酸激酶,431,U/L,H,21～190
1,28077,553406,747254,2014-11-22,血,CK,肌酸激酶,503,U/L,H,21～190
2,173988,870345,1299613,2019-11-14,血清,CK,肌酸激酶,729,U/L,H,21～190
3,186911,976792,1408468,2020-12-09,血清,CK,肌酸激酶,216,U/L,H,21～190
4,210662,934477,1346643,2020-05-26,血清,CK,肌酸激酶,326,U/L,H,21～190
...,...,...,...,...,...,...,...,...,...,...,...
50574,4000835,1024829,1462279,2021-05-27,血清,HBDH,α羟丁酸脱氢酶,363,U/L,H,72～182
50575,4005000,921071,1308611,2019-12-09,血清,HBDH,α羟丁酸脱氢酶,211,U/L,H,72～182
50576,4005828,971665,1383900,2020-09-22,血清,HBDH,α羟丁酸脱氢酶,226,U/L,H,72～182
50577,4006261,833644,1471105,2021-06-16,血清,HBDH,α羟丁酸脱氢酶,207,U/L,H,72～182


In [155]:
print(df_test_xjmp_high.shape)
print(df_test_xjmp_high['patient_id'].nunique())
print(df_test_xjmp_high['case_no'].nunique())

(23096, 11)
4591
9454


In [156]:
# 保存心肌酶谱升高的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/心脏毒性/df_temp_心肌酶谱升高.xlsx')
df_test_xjmp_high.to_excel(writer)
writer.save()

In [157]:
# 方案（二）:根据本善提供不良反应指标，磷酸肌酸激酶:CK> 194 IU/L、磷酸肌酸激酶同工酶:CK-MB> 25 IU/L、乳酸脱氢酶:LDH> 246 IU/L
# 谷草转氨酶:AST>40 IU/L、α-羟丁酸脱氢酶:α -HBDH> 182 U/L

# 磷酸肌酸激酶:CK> 194 IU/L
df_test_CK=df_test_xjmp[df_test_xjmp['project_name'].str.contains('肌酸激酶')]
df_test_CK_high=df_test_CK[df_test_CK['test_result'].astype('float')>194]
df_test_CK_high=df_test_CK_high.reset_index(drop=True)

In [158]:
print(df_test_CK_high.shape)
print(df_test_CK_high['patient_id'].nunique())
print(df_test_CK_high['case_no'].nunique())

(87, 11)
65
71


In [159]:
print(df_test_CK_high.to_string())

         id patient_id  case_no  test_date sample          project_code project_name test_result           result_unit is_normal refer_scope
0      1228     792697  1144585 2018-08-11     血清  CK                           肌酸激酶         431  U/L                          H      21～190
1     28077     553406   747254 2014-11-22      血  CK                           肌酸激酶         503  U/L                          H      21～190
2    173988     870345  1299613 2019-11-14     血清  CK                           肌酸激酶         729  U/L                          H      21～190
3    186911     976792  1408468 2020-12-09     血清  CK                           肌酸激酶         216  U/L                          H      21～190
4    210662     934477  1346643 2020-05-26     血清  CK                           肌酸激酶         326  U/L                          H      21～190
5    222163     938470  1353516 2020-06-18     血清  CK                           肌酸激酶         414  U/L                          H      21～190
6    279930  

In [160]:
# 磷酸肌酸激酶同工酶:CK-MB> 25 IU/L
df_test_CKMB=df_test_xjmp[df_test_xjmp['project_code'].str.contains('CKMB|CK-MB')]
df_test_CKMB_high=df_test_CKMB[df_test_CKMB['test_result'].astype('float')>25]
df_test_CKMB_high=df_test_CKMB_high.reset_index(drop=True)

In [161]:
print(df_test_CKMB_high.shape)
print(df_test_CKMB_high['patient_id'].nunique())
print(df_test_CKMB_high['case_no'].nunique())

(264, 11)
182
240


In [162]:
print(df_test_CKMB_high.to_string())

          id patient_id  case_no  test_date sample          project_code project_name test_result           result_unit is_normal refer_scope
0      14464     794483  1152573 2018-09-04     血清  CKMB                        CK-MB          29  U/L                          H        0～25
1      22882     547358  1077872 2018-01-16     血清  CKMB                        CK-MB          31  U/L                          H        0～25
2      28582     670951   932162 2016-09-26      血  CK-MB-DXI800                 CKMB        30.7  ug/L                         H     0.6～6.3
3      60525     933565  1359314 2020-07-08     血清  CKMB                        CK-MB          28  U/L                          H        0～25
4      99251     371834  1154011 2018-09-08     血清  CKMB                        CK-MB          40  U/L                          H        0～25
5     115271     371834  1108891 2018-04-27     血清  CKMB                        CK-MB          30  U/L                          H        0～25
6     

In [163]:
# 乳酸脱氢酶:LDH> 246 IU/L
df_test_LDH=df_test_xjmp[df_test_xjmp['project_code'].str.contains('LDH')]
df_test_LDH_high=df_test_LDH[df_test_LDH['test_result'].astype('float')>246]
df_test_LDH_high=df_test_LDH_high.reset_index(drop=True)

In [164]:
print(df_test_LDH_high.shape)
print(df_test_LDH_high['patient_id'].nunique())
print(df_test_LDH_high['case_no'].nunique())

(37222, 11)
8647
22333


In [165]:
print(df_test_LDH_high)

            id patient_id  case_no  test_date sample          project_code  \
0            6     665849   918995 2016-08-06      血  VLDH-J                 
1          205     432942   570519 2013-01-07      血  VLDH-J                 
2          389     547541   911972 2016-07-18      血  VLDH-J                 
3          571     792697  1144585 2018-08-16     血清  VLDH-J                 
4          645     495241   725374 2014-09-12      血  VLDH-J                 
...        ...        ...      ...        ...    ...                   ...   
37217  4014997     998141  1428210 2021-02-08     血浆  VLDH-J                 
37218  4015256     945291  1345066 2020-05-20     血浆  VLDH-J                 
37219  4015304     945291  1345066 2020-05-25     血浆  VLDH-J                 
37220  4015325     959410  1365759 2020-07-29     血浆  VLDH-J                 
37221  4015446     610802   835370 2015-10-21      血  VLDH-J                 

      project_name test_result           result_unit is_normal 

In [166]:
# 谷草转氨酶:AST>40 IU/L
df_test_AST=df_test_xjmp[df_test_xjmp['project_name'].str.contains('谷草转氨酶')]
df_test_AST_high=df_test_AST[df_test_AST['test_result'].astype('float')>40]
df_test_AST_high=df_test_AST_high.reset_index(drop=True)

In [167]:
print(df_test_AST_high.shape)
print(df_test_AST_high['patient_id'].nunique())
print(df_test_AST_high['case_no'].nunique())

(12152, 11)
3580
6944


In [168]:
print(df_test_AST_high.to_string())

            id patient_id  case_no  test_date sample          project_code project_name test_result           result_unit is_normal refer_scope
0          681     664611   929253 2016-09-12      血  VAST-J                  谷草转氨酶(干式)          91  U/L                          H       15～46
1          717     664611   929253 2016-09-10      血  VAST-J                  谷草转氨酶(干式)         554  U/L                          H       15～46
2         1305     974212  1387721 2020-10-13     血浆  VAST-J                  谷草转氨酶(干式)         107  U/L                          H       15～46
3         1432     493318   733984 2014-10-13      血  VAST-J                  谷草转氨酶(干式)          47  U/L                          H       15～46
4         2073     974212  1387721 2020-10-14     血浆  VAST-J                  谷草转氨酶(干式)          44  U/L                       None       15～46
5         2419     933941  1328139 2020-03-08     血浆  VAST-J                  谷草转氨酶(干式)          55  U/L                          H     

In [169]:
# α-羟丁酸脱氢酶:α -HBDH> 182 U/L
df_test_HBDH=df_test_xjmp[df_test_xjmp['project_code'].str.contains('HBDH')]
df_test_HBDH_high=df_test_HBDH[df_test_HBDH['test_result'].astype('float')>182]
df_test_HBDH_high=df_test_HBDH_high.reset_index(drop=True)

In [170]:
print(df_test_HBDH_high.shape)
print(df_test_HBDH_high['patient_id'].nunique())
print(df_test_HBDH_high['case_no'].nunique())

(854, 11)
586
831


In [171]:
print(df_test_HBDH_high.to_string())

          id patient_id  case_no  test_date sample          project_code project_name test_result           result_unit is_normal refer_scope
0       1219     792697  1144585 2018-08-11     血清  HBDH                      α羟丁酸脱氢酶         196  U/L                          H      72～182
1      15864      61036   951574 2016-11-23      血  HBDH                      α羟丁酸脱氢酶         286  U/L                          H      72～182
2      21633     936765  1359730 2020-07-10     血清  HBDH                      α羟丁酸脱氢酶         196  U/L                          H      72～182
3      22874     547358  1077872 2018-01-16     血清  HBDH                      α羟丁酸脱氢酶         216  U/L                          H      72～182
4      23337     936765  1363823 2020-07-23     血清  HBDH                      α羟丁酸脱氢酶         212  U/L                          H      72～182
5      35668     940174  1337497 2020-04-22     血清  HBDH                      α羟丁酸脱氢酶         254  U/L                          H      72～182
6     

In [172]:
df_test_xjmp_high=pd.concat([df_test_CK_high,df_test_CKMB_high],axis=0)
df_test_xjmp_high=pd.concat([df_test_xjmp_high,df_test_LDH_high],axis=0)
df_test_xjmp_high=pd.concat([df_test_xjmp_high,df_test_AST_high],axis=0)
df_test_xjmp_high=pd.concat([df_test_xjmp_high,df_test_HBDH_high],axis=0)
df_test_xjmp_high=df_test_xjmp_high.reset_index(drop=True)

In [173]:
print(df_test_xjmp_high.shape)
print(df_test_xjmp_high['patient_id'].nunique())
print(df_test_xjmp_high['case_no'].nunique())

(50579, 11)
8786
22710


In [174]:
print(df_test_xjmp_high)

            id patient_id  case_no  test_date sample          project_code  \
0         1228     792697  1144585 2018-08-11     血清  CK                     
1        28077     553406   747254 2014-11-22      血  CK                     
2       173988     870345  1299613 2019-11-14     血清  CK                     
3       186911     976792  1408468 2020-12-09     血清  CK                     
4       210662     934477  1346643 2020-05-26     血清  CK                     
...        ...        ...      ...        ...    ...                   ...   
50574  4000835    1024829  1462279 2021-05-27     血清  HBDH                   
50575  4005000     921071  1308611 2019-12-09     血清  HBDH                   
50576  4005828     971665  1383900 2020-09-22     血清  HBDH                   
50577  4006261     833644  1471105 2021-06-16     血清  HBDH                   
50578  4009616     935280  1464576 2021-05-29     血清  HBDH                   

      project_name test_result           result_unit is_normal 

In [175]:
# 保存心肌酶谱升高的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/心脏毒性/df_temp_心肌酶谱升高_本善标准.xlsx')
df_test_xjmp_high.to_excel(writer)
writer.save()

#### 提取心肌肌钙蛋白 ( cTnT, cTnI)升高的检测结果

In [176]:
df_test_CTN=df_test_result[df_test_result['project_code'].str.contains('CTN')]
df_test_CTN=df_test_CTN.reset_index(drop=True)

In [177]:
print(df_test_CTN.shape)
print(df_test_CTN['patient_id'].nunique())

(102, 11)
85


In [178]:
# 保存心肌肌钙蛋白检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/心脏毒性/df_temp_心肌肌钙蛋白.xlsx')
df_test_CTN.to_excel(writer)
writer.save()

In [179]:
# 方案（一）：提取心肌肌钙蛋白升高
df_test_CTN_high=df_test_CTN[df_test_CTN['is_normal']=='H']
df_test_CTN_high=df_test_CTN_high.reset_index(drop=True)

In [180]:
print(df_test_CTN_high.shape)
print(df_test_CTN_high['patient_id'].nunique())
print(df_test_CTN_high['case_no'].nunique())

(8, 11)
8
8


In [181]:
# 保存心肌肌钙蛋白的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/心脏毒性/df_temp_心肌肌钙蛋白升高.xlsx')
df_test_CTN_high.to_excel(writer)
writer.save()

In [182]:
# 方案（二）：高敏肌钙蛋白-T: CTNI-J>0.014ug/L
df_test_CTN_high=df_test_CTN[df_test_CTN['test_result'].astype('float')>0.014]
df_test_CTN_high=df_test_CTN_high.reset_index(drop=True)

In [183]:
print(df_test_CTN_high.shape)
print(df_test_CTN_high['patient_id'].nunique())
print(df_test_CTN_high['case_no'].nunique())

(8, 11)
8
8


In [184]:
# 保存心肌肌钙蛋白的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/心脏毒性/df_temp_心肌肌钙蛋白升高_本善标准.xlsx')
df_test_CTN_high.to_excel(writer)
writer.save()

#### 肌红蛋白升高

In [185]:
df_test_MYO=df_test_result[df_test_result['project_name'].str.contains('肌红蛋白')]
df_test_MYO=df_test_MYO.reset_index(drop=True)

In [186]:
# 保存肌红蛋白检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/心脏毒性/df_temp_肌红蛋白.xlsx')
df_test_MYO.to_excel(writer)
writer.save()

In [187]:
# 方案（一）：根据is_normal，提取肌红蛋白升高
df_test_MYO_high=df_test_MYO[df_test_MYO['is_normal']=='H']
df_test_MYO_high=df_test_MYO_high.reset_index(drop=True)

In [188]:
print(df_test_MYO_high.shape)
print(df_test_MYO_high['patient_id'].nunique())
print(df_test_MYO_high['case_no'].nunique())

(112, 11)
71
72


In [189]:
# 保存肌红蛋白升高检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/心脏毒性/df_temp_肌红蛋白升高.xlsx')
df_test_MYO_high.to_excel(writer)
writer.save()

In [190]:
# 并入性别信息
df_test_MYO_gender=pd.merge(df_test_MYO,df_popu,on=['patient_id'],how='left')

In [191]:
# 方案（二）:根据肌红蛋白:男性Mb>105.7，女性Mb>65.8，提取肌红蛋白升高
# 男性Mb>105.7
df_test_MYO_male=df_test_MYO_gender[df_test_MYO_gender['gender'].str.contains('男')]
df_test_MYO_male_high=df_test_MYO_male[df_test_MYO_male['test_result'].astype('float')>105.7]
df_test_MYO_male_high=df_test_MYO_male_high.reset_index(drop=True)

In [192]:
# 女性Mb>65.8
df_test_MYO_female=df_test_MYO_gender[df_test_MYO_gender['gender'].str.contains('女')]
df_test_MYO_female_high=df_test_MYO_female[df_test_MYO_female['test_result'].astype('float')>105.7]
df_test_MYO_female_high=df_test_MYO_female_high.reset_index(drop=True)

In [193]:
df_test_MYO_high=pd.concat([df_test_MYO_male_high,df_test_MYO_female_high],axis=0)
df_test_MYO_high=df_test_MYO_high.reset_index(drop=True)

In [194]:
print(df_test_MYO_high.shape)
print(df_test_MYO_high['patient_id'].nunique())
print(df_test_MYO_high['case_no'].nunique())

(107, 13)
68
69


In [195]:
# 保存肌红蛋白升高检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/心脏毒性/df_temp_肌红蛋白升高_本善标准.xlsx')
df_test_MYO_high.to_excel(writer)
writer.save()

#### 合并心肌酶谱、心肌肌钙蛋白、肌红蛋白升高检测记录

In [196]:
# 合并心脏毒性所有检测
df_test_heart_toxicity=pd.concat([df_test_xjmp,df_test_CTN],axis=0)
df_test_heart_toxicity=pd.concat([df_test_heart_toxicity,df_test_MYO],axis=0)

In [197]:
# 排序
df_test_heart_toxicity=df_test_heart_toxicity.sort_values(['patient_id','test_date'])
df_test_heart_toxicity=df_test_heart_toxicity.reset_index(drop=True)

In [198]:
print(df_test_heart_toxicity.shape)
print(df_test_heart_toxicity['patient_id'].nunique())
print(df_test_heart_toxicity['case_no'].nunique())

(136736, 11)
10377
27421


In [199]:
# 保存骨髓抑制的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_2.2.1_心脏毒性所有检测.xlsx')
df_test_heart_toxicity.to_excel(writer)
writer.save()

In [200]:
df_test_heart_toxicity_high=pd.concat([df_test_xjmp_high,df_test_CTN_high],axis=0)
df_test_heart_toxicity_high=pd.concat([df_test_heart_toxicity_high,df_test_MYO_high],axis=0)
df_test_heart_toxicity_high=df_test_heart_toxicity_high.reset_index(drop=True)

In [201]:
print(df_test_heart_toxicity_high.shape)
print(df_test_heart_toxicity_high['patient_id'].nunique())
print(df_test_heart_toxicity_high['case_no'].nunique())

(50694, 13)
8787
22711


In [202]:
# 保存心脏毒性检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_2.2.2_心脏毒性指标升高的所有检测.xlsx')
df_test_heart_toxicity_high.to_excel(writer)
writer.save()

### 提取骨髓抑制相关检测

#### 提取血红蛋白（g/L）<109

In [92]:
df_test_HGB=df_test_result[(df_test_result['project_name'].str.contains('血红蛋白')) & (df_test_result['project_code'].str.contains('HGB'))]
df_test_HGB=df_test_HGB.reset_index(drop=True)

In [93]:
# 排序
df_test_HGB=df_test_HGB.sort_values(['patient_id','test_date'])

In [94]:
# 保存血红蛋白检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/骨髓抑制/df_temp_血红蛋白.xlsx')
df_test_HGB.to_excel(writer)
writer.save()

In [95]:
# # 方案（一）：根据is_normal指标，提取血红蛋白较低
# df_test_HGB_low=df_test_HGB[df_test_HGB['is_normal']=='L']

In [96]:
# 方案（二）：提取血红蛋白（g/L）<109
df_test_HGB_low=df_test_HGB[df_test_HGB['test_result'].astype('int')<=109]

In [97]:
# 排序
df_test_HGB_low=df_test_HGB_low.sort_values(['patient_id','test_date'],ascending=[True,True])
df_test_HGB_low=df_test_HGB_low.reset_index(drop=True)
df_test_HGB_low

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
0,3800330,1000034,1425903,2021-02-01,全血,HGB,血红蛋白,109,g/L,L,115～150
1,3889104,1000106,1453657,2021-05-02,全血,HGB,血红蛋白,100,g/L,L,130～175
2,3869235,1000106,1462565,2021-05-24,全血,HGB,血红蛋白,96,g/L,L,130～175
3,3872157,1000106,1462565,2021-05-26,全血,HGB,血红蛋白,98,g/L,L,130～175
4,3963426,1000106,1462565,2021-05-28,全血,HGB,血红蛋白,99,g/L,L,130～175
...,...,...,...,...,...,...,...,...,...,...,...
20862,3541695,999759,1425458,2021-02-03,全血,HGB,血红蛋白,96,g/L,L,115～150
20863,3752209,999933,1425736,2021-02-05,全血,HGB,血红蛋白,108,g/L,L,115～150
20864,3753122,999933,1425736,2021-02-07,全血,HGB,血红蛋白,91,g/L,L,115～150
20865,3753149,999933,1425736,2021-02-08,全血,HGB,血红蛋白,91,g/L,L,115～150


In [98]:
print(df_test_HGB_low.shape)
print(df_test_HGB_low['patient_id'].nunique())
print(df_test_HGB_low['case_no'].nunique())

(20867, 11)
4954
10428


In [99]:
# 保存血红蛋白（g/L）<109的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/骨髓抑制/df_temp_血红蛋白较低.xlsx')
df_test_HGB_low.to_excel(writer)
writer.save()

#### 提取白细胞（109/L）<3.9

In [100]:
# 白细胞（109/L）<3.9
df_test_WBC=df_test_result[(df_test_result['project_name']=='白细胞') & (df_test_result['sample'].str.contains('血')) ]
df_test_WBC=df_test_WBC.reset_index(drop=True)

In [101]:
# 保存血液白细胞检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/骨髓抑制/df_temp_白细胞.xlsx')
df_test_WBC.to_excel(writer)
writer.save()

In [102]:
# # 方案（一）：根据is_normal字段，提取白细胞检测偏低
# df_test_WBC_low=df_test_WBC[df_test_WBC['is_normal']=='L']

In [103]:
# 方案（二）：提取白细胞（109/L）<3.9
df_test_WBC_low=df_test_WBC[df_test_WBC['test_result'].astype('float')<=3.9]

In [104]:
# 排序
df_test_WBC_low=df_test_WBC_low.sort_values(['patient_id','test_date'],ascending=[True,True])
df_test_WBC_low=df_test_WBC_low.reset_index(drop=True)
df_test_WBC_low

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
0,3801413,1000034,1425903,2021-02-03,全血,WBC,白细胞,3.6,X10^9/L,None,3.5～9.5
1,3804465,1000034,1425903,2021-02-04,全血,WBC,白细胞,3.7,X10^9/L,None,3.5～9.5
2,561584,1000106,1425998,2021-02-01,全血,WBC,白细胞,3.9,*10^9/L,None,3.5～9.5
3,3869228,1000106,1462565,2021-05-24,全血,WBC,白细胞,3.7,*10^9/L,None,3.5～9.5
4,3811037,1000106,1476889,2021-07-04,全血,WBC,白细胞,3.8,X10^9/L,None,3.5～9.5
...,...,...,...,...,...,...,...,...,...,...,...
11971,3759088,999595,1425207,2021-01-31,全血,WBC,白细胞,2.7,X10^9/L,L,3.5～9.5
11972,3652857,999601,1455226,2021-05-03,全血,WBC,白细胞,3.9,X10^9/L,None,3.5～9.5
11973,3838168,999620,1425253,2021-01-29,全血,WBC,白细胞,3.9,X10^9/L,None,3.5～9.5
11974,3538529,999759,1425458,2021-01-29,全血,WBC,白细胞,3.9,X10^9/L,None,3.5～9.5


In [105]:
print(df_test_WBC_low.shape)
print(df_test_WBC_low['patient_id'].nunique())
print(df_test_WBC_low['case_no'].nunique())

(11976, 11)
4041
9133


In [106]:
# 保存白细胞（109/L）<3.9的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/骨髓抑制/df_temp_白细胞较低.xlsx')
df_test_WBC_low.to_excel(writer)
writer.save()

#### 提取粒细胞（109/L）<1.9

In [107]:
# 粒细胞（109/L）<1.9
df_test_IG=df_test_result[(df_test_result['project_code'].str.contains('IG#')) & (df_test_result['sample'].str.contains('血')) ]
df_test_IG=df_test_IG.reset_index(drop=True)

In [108]:
# 保存粒细胞检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/骨髓抑制/df_temp_粒细胞.xlsx')
df_test_IG.to_excel(writer)
writer.save()

In [689]:
# # 方案（一）：根据is_normal指标，提取粒细胞较低
# df_test_IG_low=df_test_IG[df_test_IG['is_normal']=='L']

In [109]:
# 方案（二）：提取粒细胞（109/L）<1.9
df_test_IG_low=df_test_IG[df_test_IG['test_result'].astype('float')<=1.9]

In [110]:
# 排序
df_test_IG_low=df_test_IG_low.sort_values(['patient_id','test_date'],ascending=[True,True])
df_test_IG_low=df_test_IG_low.reset_index(drop=True)
df_test_IG_low

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
0,502920,1000000,1425855,2021-01-31,全血,IG#,幼稚粒细胞绝对值,0.03,*10^9/L,None,0.00～0.20
1,504419,1000000,1425855,2021-02-04,全血,IG#,幼稚粒细胞绝对值,0.04,*10^9/L,None,0.00～0.20
2,717123,1000034,1425903,2021-01-31,全血,IG#,幼稚粒细胞绝对值,0.02,*10^9/L,None,0.00～0.20
3,561592,1000106,1425998,2021-02-01,全血,IG#,幼稚粒细胞绝对值,0.01,X10^9/L,None,0.00～0.20
4,3888422,1000106,1425998,2021-02-03,全血,IG#,幼稚粒细胞绝对值,0.03,*10^9/L,None,0.00～0.20
...,...,...,...,...,...,...,...,...,...,...,...
38070,1998739,999935,1425739,2021-02-01,全血,IG#,幼稚粒细胞绝对值,0.02,X10^9/L,None,0.00～0.20
38071,3745619,999935,1430932,2021-02-23,全血,IG#,幼稚粒细胞绝对值,0.01,X10^9/L,None,0.00～0.20
38072,3748179,999935,1430932,2021-02-26,全血,IG#,幼稚粒细胞绝对值,0.04,*10^9/L,None,0.00～0.20
38073,3749452,999935,1430932,2021-02-28,全血,IG#,幼稚粒细胞绝对值,0.02,*10^9/L,None,0.00～0.20


In [111]:
print(df_test_IG_low.shape)
print(df_test_IG_low['patient_id'].nunique())
print(df_test_IG_low['case_no'].nunique())

(38075, 11)
9900
23888


In [112]:
# 保存粒细胞（109/L）<1.9的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/骨髓抑制/df_temp_粒细胞较低.xlsx')
df_test_IG_low.to_excel(writer)
writer.save()

#### 血小板（109/L）<99

In [113]:
# 血小板（109/L）<99
df_test_PLT=df_test_result[(df_test_result['project_code'].str.contains('PLT')) & (df_test_result['sample'].str.contains('血')) ]
df_test_PLT=df_test_PLT.reset_index(drop=True)

In [114]:
# 删除文字检测结果
df_test_PLT=filter_string(df_test_PLT,'test_result')

In [115]:
df_test_PLT[df_test_PLT['case_no']=='786473']

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
22504,1479268,563831,786473,2015-04-23,血,PLT,血小板,141,X10^9/L,None,125～350
22531,1481386,563831,786473,2015-04-21,血,PLT,血小板,145,X10^9/L,None,125～350


In [116]:
# 保存血小板检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/骨髓抑制/df_temp_血小板.xlsx')
df_test_PLT.to_excel(writer)
writer.save()

In [721]:
# # 方案（一）：根据is_normal，提取血小板较低
# df_test_PLT_low=df_test_PLT[df_test_PLT['is_normal']=='L']

In [117]:
# 方案（二）：提取血小板（109/L）<99
df_test_PLT_low=df_test_PLT[df_test_PLT['test_result'].astype('float')<=99]

In [118]:
# 排序
df_test_PLT_low=df_test_PLT_low.sort_values(['patient_id','test_date'],ascending=[True,True])
df_test_PLT_low=df_test_PLT_low.reset_index(drop=True)
df_test_PLT_low

,id,patient_id,case_no,test_date,sample,project_code,project_name,test_result,result_unit,is_normal,refer_scope
0,3337396,1000123,1428796,2021-02-12,全血,PLT,血小板,92,X10^9/L,L,125～350
1,3339117,1000123,1428796,2021-02-13,全血,PLT,血小板,60,X10^9/L,L,125～350
2,3339133,1000123,1428796,2021-02-14,全血,PLT,血小板,86,X10^9/L,L,125～350
3,3793554,1002219,1429179,2021-02-15,全血,PLT,血小板,90,X10^9/L,L,125～350
4,4015202,1002462,1470779,2021-06-16,全血,PLT,血小板,95,X10^9/L,L,125～350
...,...,...,...,...,...,...,...,...,...,...,...
2317,3867981,999385,1424896,2021-06-02,全血,PLT,血小板,93,X10^9/L,L,125～350
2318,3867997,999385,1424896,2021-06-03,全血,PLT,血小板,61,X10^9/L,L,125～350
2319,3868014,999385,1424896,2021-06-04,全血,PLT,血小板,56,X10^9/L,L,125～350
2320,3868846,999385,1424896,2021-06-05,全血,PLT,血小板,40,X10^9/L,L,125～350


In [119]:
print(df_test_PLT_low.shape)
print(df_test_PLT_low['patient_id'].nunique())
print(df_test_PLT_low['case_no'].nunique())

(2322, 11)
767
1057


In [120]:
# 保存血小板（109/L）<99的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/骨髓抑制/df_temp_血小板较低.xlsx')
df_test_PLT_low.to_excel(writer)
writer.save()

#### 合并血红蛋白、白细胞、粒细胞、血小板较低检测结果

In [121]:
# 合并骨髓抑制检测
df_test_bmd=pd.concat([df_test_HGB,df_test_WBC],axis=0)
df_test_bmd=pd.concat([df_test_bmd,df_test_IG],axis=0)
df_test_bmd=pd.concat([df_test_bmd,df_test_PLT],axis=0)

In [122]:
# 排序
df_test_bmd=df_test_bmd.sort_values(['patient_id','test_date'])
df_test_bmd=df_test_bmd.reset_index(drop=True)

In [123]:
print(df_test_bmd.shape)
print(df_test_bmd['patient_id'].nunique())
print(df_test_bmd['case_no'].nunique())

(217767, 11)
10232
27157


In [124]:
# 保存骨髓抑制的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_2.3.1_骨髓抑制所有检测.xlsx')
df_test_bmd.to_excel(writer)
writer.save()

In [125]:
# 合并血红蛋白、白细胞、粒细胞、血小板较低检测结果
df_test_bmd_low=pd.concat([df_test_HGB_low,df_test_WBC_low],axis=0)
df_test_bmd_low=pd.concat([df_test_bmd_low,df_test_IG_low],axis=0)
df_test_bmd_low=pd.concat([df_test_bmd_low,df_test_PLT_low],axis=0)

In [126]:
# 排序
df_test_bmd_low=df_test_bmd_low.sort_values(['patient_id','test_date'])
df_test_bmd_low=df_test_bmd_low.reset_index(drop=True)

In [127]:
print(df_test_bmd_low.shape)
print(df_test_bmd_low['patient_id'].nunique())
print(df_test_bmd_low['case_no'].nunique())

(73240, 11)
10078
25719


In [128]:
# 保存骨髓抑制的检测结果
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_2.3.2_骨髓抑制指标降低的所有检测.xlsx')
df_test_bmd_low.to_excel(writer)
writer.save()

# 添加人口学信息

In [129]:
df_MTX_popu=pd.merge(df_MTX,df_popu,on=['patient_id'],how='left')

In [130]:
print(df_MTX_popu.shape)
print(df_MTX_popu['patient_id'].nunique())

(1321, 13)
718


In [131]:
df_MTX_popu

,patient_id,case_no,long_d_order,drug_name,amount,drug_spec,dosage,frequency,medication_way,start_datetime,end_datetime,gender,age
0,175391,601919,长期医嘱,(甲)注射用甲氨蝶呤（国产）,13.0,1g/瓶,13.000,QD,静滴,2013-05-17 11:21:01,2013-05-18 09:01:01,男,51
1,283828,352356,长期医嘱,(甲)甲氨喋呤注射液（MTX）,13.0,1g*10ml*1瓶/瓶,13.000,QD,静滴,2010-04-26 11:28:00,2010-04-27 11:28:00,男,19
2,289182,364846,长期医嘱,(甲)甲氨喋呤注射液（MTX）,13.0,1g*10ml*1瓶/瓶,13.000,QD,静滴,2010-06-28 10:31:00,2010-06-29 10:31:00,男,18
3,292204,361726,长期医嘱,(甲)甲氨喋呤注射液（MTX）,16.0,1g*10ml*1瓶/瓶,16.000,QD,静滴,2010-06-29 07:54:00,2010-06-30 07:54:00,男,20
4,292204,366167,长期医嘱,(甲)甲氨喋呤注射液（MTX）,14.0,1g*10ml*1瓶/瓶,14.000,QD,静滴,2010-07-13 10:17:00,2010-07-14 10:17:00,男,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,865779,1225981,长期医嘱,(甲)甲氨喋呤注射液（进口）,15.0,1g*10ml/瓶,15.000,QD,静滴,2019-04-16 14:36:19,2019-04-17 14:36:00,男,14
1317,866260,1226693,长期医嘱,(甲)甲氨喋呤注射液（进口）,20.0,1g*10ml/瓶,20.000,QD,静滴,2019-04-18 09:47:24,2019-04-19 09:47:00,男,22
1318,866481,1227007,长期医嘱,(甲)甲氨喋呤注射液（进口）,15.0,1g*10ml/瓶,15.000,QD,静滴,2019-04-19 10:20:07,2019-04-20 10:20:08,女,22
1319,868362,1229838,长期医嘱,(甲)甲氨喋呤注射液（进口）,16.0,1g*10ml/瓶,16.000,QD,静滴,2019-04-27 09:00:00,2019-04-28 07:40:38,男,15


# 计算甲氨蝶呤日剂量

In [132]:
df_MTX_dosage=df_MTX_popu.drop(['long_d_order','amount','drug_spec','frequency','medication_way'],axis=1)
df_MTX_dosage.rename(columns={'dosage':'日剂量'},inplace=True)

In [133]:
print(df_MTX_dosage.shape)
print(df_MTX_dosage['patient_id'].nunique())

(1321, 8)
718


In [135]:
# 保存甲氨蝶呤日剂量
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_4_计算甲氨蝶呤日剂量.xlsx')
df_MTX_dosage.to_excel(writer)
writer.save()

# 提取甲氨蝶呤的骨髓抑制信息

## 骨髓抑制检测转置

In [136]:
# 扩展新列
df_MTX_dosage.loc[:,'血红蛋白_before']=np.nan
df_MTX_dosage.loc[:,'血红蛋白_3d']=np.nan
df_MTX_dosage.loc[:,'血红蛋白_7d']=np.nan
df_MTX_dosage.loc[:,'血红蛋白_14d']=np.nan
df_MTX_dosage.loc[:,'白细胞_before']=np.nan
df_MTX_dosage.loc[:,'白细胞_3d']=np.nan
df_MTX_dosage.loc[:,'白细胞_7d']=np.nan
df_MTX_dosage.loc[:,'白细胞_14d']=np.nan
df_MTX_dosage.loc[:,'粒细胞_before']=np.nan
df_MTX_dosage.loc[:,'粒细胞_3d']=np.nan
df_MTX_dosage.loc[:,'粒细胞_7d']=np.nan
df_MTX_dosage.loc[:,'粒细胞_14d']=np.nan
df_MTX_dosage.loc[:,'血小板_before']=np.nan
df_MTX_dosage.loc[:,'血小板_3d']=np.nan
df_MTX_dosage.loc[:,'血小板_7d']=np.nan
df_MTX_dosage.loc[:,'血小板_14d']=np.nan

In [137]:
df_MTX_dosage.shape

(1321, 24)

In [170]:
# 将一次用药的不同检测装置到一行中，比如WBC_before,WBC_3d,WBC_7d
temp_list=[]
for i in df_MTX_dosage['case_no'].unique():
    temp=df_MTX_dosage[df_MTX_dosage['case_no']==i]
    temp=temp.sort_values(['case_no','start_datetime'])
    temp=temp.reset_index(drop=True)
    # 血红蛋白检测
    temp_HGB=df_test_HGB[df_test_HGB['case_no']==i]
    temp_HGB=temp_HGB.sort_values(['case_no','test_date'])
    temp_HGB=temp_HGB.reset_index(drop=True)
    # 白细胞检测
    temp_WBC=df_test_WBC[df_test_WBC['case_no']==i]
    temp_WBC=temp_WBC.sort_values(['case_no','test_date'])
    temp_WBC=temp_WBC.reset_index(drop=True)
    # 粒细胞检测
    temp_IG=df_test_IG[df_test_IG['case_no']==i]
    temp_IG=temp_IG.sort_values(['case_no','test_date'])
    temp_IG=temp_IG.reset_index(drop=True)
    # 血小板检测
    temp_PLT=df_test_PLT[df_test_PLT['case_no']==i]
    temp_PLT=temp_PLT.sort_values(['case_no','test_date'])
    temp_PLT=temp_PLT.reset_index(drop=True)
    # 骨髓抑制是降低指标，应该取用药后第一次检测。
    for j in range(temp.shape[0]):
        # 血红蛋白检测转置
        for k in range(temp_HGB.shape[0]):
            x=(temp_HGB.loc[k,'test_date']-temp.loc[j,'start_datetime']).days
            if (x>=-3)&(x<0):
                temp.loc[j,'血红蛋白_before']=temp_HGB.loc[k,'test_result']
            if (x>=1)&(x<=4):
                if pd.isnull(temp.loc[j,'血红蛋白_3d']):
                    temp.loc[j,'血红蛋白_3d']=temp_HGB.loc[k,'test_result']
            if (x>=6)&(x<=8):
                if pd.isnull(temp.loc[j,'血红蛋白_7d']):
                    temp.loc[j,'血红蛋白_7d']=temp_HGB.loc[k,'test_result']
            if (x>=13)&(x<=15):
                if pd.isnull(temp.loc[j,'血红蛋白_14d']):
                    temp.loc[j,'血红蛋白_14d']=temp_HGB.loc[k,'test_result']
        # 白细胞检测转置
        for k in range(temp_WBC.shape[0]):
            x=(temp_WBC.loc[k,'test_date']-temp.loc[j,'start_datetime']).days
            if (x>=-3)&(x<0):
                temp.loc[j,'白细胞_before']=temp_WBC.loc[k,'test_result']
            if (x>=1)&(x<=4):
                if pd.isnull(temp.loc[j,'白细胞_3d']):
                    temp.loc[j,'白细胞_3d']=temp_WBC.loc[k,'test_result']
            if (x>=6)&(x<=8):
                if pd.isnull(temp.loc[j,'白细胞_7d']):
                    temp.loc[j,'白细胞_7d']=temp_WBC.loc[k,'test_result']
            if (x>=13)&(x<=15):
                if pd.isnull(temp.loc[j,'白细胞_14d']):
                    temp.loc[j,'白细胞_14d']=temp_WBC.loc[k,'test_result']
        # 粒细胞检测转置
        for k in range(temp_IG.shape[0]):
            x=(temp_IG.loc[k,'test_date']-temp.loc[j,'start_datetime']).days
            if (x>=-3)&(x<0):
                temp.loc[j,'粒细胞_before']=temp_IG.loc[k,'test_result']
            if (x>=1)&(x<=4):
                if pd.isnull(temp.loc[j,'粒细胞_3d']):
                    temp.loc[j,'粒细胞_3d']=temp_IG.loc[k,'test_result']
            if (x>=6)&(x<=8):
                if pd.isnull(temp.loc[j,'粒细胞_7d']):
                    temp.loc[j,'粒细胞_7d']=temp_IG.loc[k,'test_result']
            if (x>=13)&(x<=15):
                if pd.isnull(temp.loc[j,'粒细胞_14d']):
                    temp.loc[j,'粒细胞_14d']=temp_IG.loc[k,'test_result']
        # 血小板检测转置
        for k in range(temp_PLT.shape[0]):
            x=(temp_PLT.loc[k,'test_date']-temp.loc[j,'start_datetime']).days
            if (x>=-3)&(x<0):
                temp.loc[j,'血小板_before']=temp_PLT.loc[k,'test_result']
            if (x>=1)&(x<=4):
                if pd.isnull(temp.loc[j,'血小板_3d']):
                    temp.loc[j,'血小板_3d']=temp_PLT.loc[k,'test_result']
            if (x>=6)&(x<=8):
                if pd.isnull(temp.loc[j,'血小板_7d']):
                    temp.loc[j,'血小板_7d']=temp_PLT.loc[k,'test_result']
            if (x>=13)&(x<=15):
                if pd.isnull(temp.loc[j,'血小板_14d']):
                    temp.loc[j,'血小板_14d']=temp_PLT.loc[k,'test_result']
    temp_list.append(temp)

In [171]:
df_MTX_bmd_test_T=temp_list[0]
for j in range(1,len(temp_list)):
    df_MTX_bmd_test_T=pd.concat([df_MTX_bmd_test_T,temp_list[j]],axis=0)
df_MTX_bmd_test_T=df_MTX_bmd_test_T.reset_index(drop=True)
del temp_list

In [172]:
df_MTX_bmd_test_T[df_MTX_bmd_test_T['case_no']=='786473']

,patient_id,case_no,drug_name,日剂量,start_datetime,end_datetime,gender,age,血红蛋白_before,血红蛋白_3d,...,白细胞_7d,白细胞_14d,粒细胞_before,粒细胞_3d,粒细胞_7d,粒细胞_14d,血小板_before,血小板_3d,血小板_7d,血小板_14d
520,563831,786473,(甲)甲氨喋呤注射液（进口）,14.000,2015-04-21 11:48:59,2015-04-22 09:01:01,女,10,119,118,...,NaN,NaN,0.05,NaN,NaN,NaN,145,141,NaN,NaN


In [174]:
# 排序
df_MTX_bmd_test_T=df_MTX_bmd_test_T.sort_values(['case_no','start_datetime'])
df_MTX_bmd_test_T=df_MTX_bmd_test_T.reset_index(drop=True)

In [175]:
print(df_MTX_bmd_test_T.shape)
print(df_MTX_bmd_test_T['case_no'].nunique())

(1321, 24)
1321


In [176]:
# 保存甲氨蝶呤骨髓抑制检测
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.1_甲氨蝶呤骨髓抑制检测转置.xlsx')
df_MTX_bmd_test_T.to_excel(writer)
writer.save()

## 骨髓抑制检测值0-1转换

In [318]:
# 根据本善提供的指标对骨髓抑制指标进行二进制转换
df_MTX_bmd_test_binary=df_MTX_bmd_test_T.copy()
# 判断血红蛋白指标
df_MTX_bmd_test_binary['血红蛋白_before_B']=df_MTX_bmd_test_binary['血红蛋白_before'].apply(lambda x: x if pd.isnull(x) else
                                                                                  1 if float(x)<=109 else 0)
df_MTX_bmd_test_binary['血红蛋白_3d_B']=df_MTX_bmd_test_binary['血红蛋白_3d'].apply(lambda x: x if pd.isnull(x) else
                                                                                  1 if float(x)<=109 else 0)
df_MTX_bmd_test_binary['血红蛋白_7d_B']=df_MTX_bmd_test_binary['血红蛋白_7d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=109 else 0)
df_MTX_bmd_test_binary['血红蛋白_14d_B']=df_MTX_bmd_test_binary['血红蛋白_14d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=109 else 0)
# 判断白细胞指标
df_MTX_bmd_test_binary['白细胞_before_B']=df_MTX_bmd_test_binary['白细胞_before'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=3.9 else 0)
df_MTX_bmd_test_binary['白细胞_3d_B']=df_MTX_bmd_test_binary['白细胞_3d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=3.9 else 0)
df_MTX_bmd_test_binary['白细胞_7d_B']=df_MTX_bmd_test_binary['白细胞_7d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=3.9 else 0)
df_MTX_bmd_test_binary['白细胞_14d_B']=df_MTX_bmd_test_binary['白细胞_14d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=3.9 else 0)
# 判断粒细胞指标
df_MTX_bmd_test_binary['粒细胞_before_B']=df_MTX_bmd_test_binary['粒细胞_before'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=1.9 else 0)
df_MTX_bmd_test_binary['粒细胞_3d_B']=df_MTX_bmd_test_binary['粒细胞_3d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=1.9 else 0)
df_MTX_bmd_test_binary['粒细胞_7d_B']=df_MTX_bmd_test_binary['粒细胞_7d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=1.9 else 0)
df_MTX_bmd_test_binary['粒细胞_14d_B']=df_MTX_bmd_test_binary['粒细胞_14d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=1.9 else 0)
# 判断血小板指标
df_MTX_bmd_test_binary['血小板_before_B']=df_MTX_bmd_test_binary['血小板_before'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=99 else 0)
df_MTX_bmd_test_binary['血小板_3d_B']=df_MTX_bmd_test_binary['血小板_3d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=99 else 0)
df_MTX_bmd_test_binary['血小板_7d_B']=df_MTX_bmd_test_binary['血小板_7d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=99 else 0)
df_MTX_bmd_test_binary['血小板_14d_B']=df_MTX_bmd_test_binary['血小板_14d'].apply(lambda x: x if pd.isnull(x) else
                                                                              1 if float(x)<=99 else 0)

In [319]:
df_MTX_bmd_test_binary

,patient_id,case_no,drug_name,日剂量,start_datetime,end_datetime,gender,age,血红蛋白_before,血红蛋白_3d,...,白细胞_7d_B,白细胞_14d_B,粒细胞_before_B,粒细胞_3d_B,粒细胞_7d_B,粒细胞_14d_B,血小板_before_B,血小板_3d_B,血小板_7d_B,血小板_14d_B
0,678710,1000051,(甲)注射用甲氨蝶呤（国产）,18.000,2017-05-11 14:52:36,2017-05-12 14:52:37,女,30,109,115,...,NaN,NaN,1.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN
1,708427,1000527,(甲)注射用甲氨蝶呤（国产）,20.000,2017-05-11 09:37:56,2017-05-12 09:38:00,男,45,119,118,...,NaN,NaN,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN
2,705404,1001400,(甲)注射用甲氨蝶呤（国产）,16.000,2017-05-15 09:01:12,2017-05-16 09:00:00,男,14,122,122,...,NaN,NaN,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN
3,690432,1001546,(甲)注射用甲氨蝶呤（国产）,14.000,2017-05-15 09:00:00,2017-05-16 08:26:00,男,11,116,110,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
4,716582,1002666,(甲)注射用甲氨蝶呤（国产）,11.000,2017-05-18 10:12:14,2017-05-19 10:12:14,女,9,127,117,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,648655,998237,(甲)注射用甲氨蝶呤（国产）,17.000,2017-05-05 09:00:00,2017-05-06 07:58:00,男,21,146,148,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1317,709392,998241,(甲)注射用甲氨蝶呤（国产）,12.400,2017-05-04 09:00:00,2017-05-05 07:15:00,男,12,NaN,123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1318,711099,999207,(甲)甲氨喋呤注射液（进口）,14.000,2017-05-08 09:00:00,2017-05-09 08:21:00,女,15,126,120,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1319,620655,999324,(甲)注射用甲氨蝶呤（国产）,16.000,2017-05-09 15:39:14,2017-05-10 15:38:00,女,37,75,74,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN


In [320]:
print(df_MTX_bmd_test_binary.shape)
print(df_MTX_bmd_test_binary['patient_id'].nunique())

(1321, 40)
718


In [321]:
# 保存甲氨蝶呤骨髓抑制检测
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.2_甲氨蝶呤骨髓抑制检测0-1转置.xlsx')
df_MTX_bmd_test_binary.to_excel(writer)
writer.save()

## 判断骨髓抑制

In [322]:
# 如果同时出现2个指标降低，则认为发生
for i in range(df_MTX_bmd_test_binary.shape[0]):
    # 检测甲氨蝶呤服药前骨髓抑制
    list_before=list(df_MTX_bmd_test_binary.loc[i,['血红蛋白_before_B','白细胞_before_B','粒细胞_before_B','血小板_before_B']])
    if list_before.count(1) >=2:
        df_MTX_bmd_test_binary.loc[i,'bmd_before']=1
    else:
        df_MTX_bmd_test_binary.loc[i,'bmd_before']=0
    # 检测甲氨蝶呤服药第3天骨髓抑制
    list_before=list(df_MTX_bmd_test_binary.loc[i,['血红蛋白_3d_B','白细胞_3d_B','粒细胞_3d_B','血小板_3d_B']])
    if list_before.count(1) >=2:
        df_MTX_bmd_test_binary.loc[i,'bmd_3d']=1
    else:
        df_MTX_bmd_test_binary.loc[i,'bmd_3d']=0
    # 检测甲氨蝶呤服药第7天骨髓抑制
    list_before=list(df_MTX_bmd_test_binary.loc[i,['血红蛋白_7d_B','白细胞_7d_B','粒细胞_7d_B','血小板_7d_B']])
    if list_before.count(1) >=2:
        df_MTX_bmd_test_binary.loc[i,'bmd_7d']=1
    else:
        df_MTX_bmd_test_binary.loc[i,'bmd_7d']=0
    # 检测甲氨蝶呤服药第14天骨髓抑制
    list_before=list(df_MTX_bmd_test_binary.loc[i,['血红蛋白_14d_B','白细胞_14d_B','粒细胞_14d_B','血小板_14d_B']])
    if list_before.count(1) >=2:
        df_MTX_bmd_test_binary.loc[i,'bmd_14d']=1
    else:
        df_MTX_bmd_test_binary.loc[i,'bmd_14d']=0

In [323]:
df_MTX_bmd_test_binary

,patient_id,case_no,drug_name,日剂量,start_datetime,end_datetime,gender,age,血红蛋白_before,血红蛋白_3d,...,粒细胞_7d_B,粒细胞_14d_B,血小板_before_B,血小板_3d_B,血小板_7d_B,血小板_14d_B,bmd_before,bmd_3d,bmd_7d,bmd_14d
0,678710,1000051,(甲)注射用甲氨蝶呤（国产）,18.000,2017-05-11 14:52:36,2017-05-12 14:52:37,女,30,109,115,...,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0
1,708427,1000527,(甲)注射用甲氨蝶呤（国产）,20.000,2017-05-11 09:37:56,2017-05-12 09:38:00,男,45,119,118,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0
2,705404,1001400,(甲)注射用甲氨蝶呤（国产）,16.000,2017-05-15 09:01:12,2017-05-16 09:00:00,男,14,122,122,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0
3,690432,1001546,(甲)注射用甲氨蝶呤（国产）,14.000,2017-05-15 09:00:00,2017-05-16 08:26:00,男,11,116,110,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0
4,716582,1002666,(甲)注射用甲氨蝶呤（国产）,11.000,2017-05-18 10:12:14,2017-05-19 10:12:14,女,9,127,117,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,648655,998237,(甲)注射用甲氨蝶呤（国产）,17.000,2017-05-05 09:00:00,2017-05-06 07:58:00,男,21,146,148,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0
1317,709392,998241,(甲)注射用甲氨蝶呤（国产）,12.400,2017-05-04 09:00:00,2017-05-05 07:15:00,男,12,NaN,123,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0
1318,711099,999207,(甲)甲氨喋呤注射液（进口）,14.000,2017-05-08 09:00:00,2017-05-09 08:21:00,女,15,126,120,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0
1319,620655,999324,(甲)注射用甲氨蝶呤（国产）,16.000,2017-05-09 15:39:14,2017-05-10 15:38:00,女,37,75,74,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0


In [324]:
# 排序
df_MTX_bmd_test_binary=df_MTX_bmd_test_binary.sort_values(['patient_id','start_datetime'])
df_MTX_bmd_test_binary=df_MTX_bmd_test_binary.reset_index(drop=True)

In [325]:
# 保存甲氨蝶呤骨髓抑制统计
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.3_判断是否出现骨髓抑制.xlsx')
df_MTX_bmd_test_binary.to_excel(writer)
writer.save()

## 提取第一次入院第一次出现骨髓抑制

### 提取第一次入院时的骨髓抑制

In [326]:
# 取第一次入院时的用药信息
df_MTX_bmd_test_binary=df_MTX_bmd_test_binary.sort_values(['patient_id','start_datetime'])
df_MTX_bmd_test_first=df_MTX_bmd_test_binary.drop_duplicates(subset=['patient_id'],keep='first')
df_MTX_bmd_test_first=df_MTX_bmd_test_first.reset_index(drop=True)

In [327]:
print(df_MTX_bmd_test_first.shape)
print(df_MTX_bmd_test_first['patient_id'].nunique())

(718, 44)
718


In [328]:
# 保存第一次入院时甲氨蝶呤骨髓抑制统计
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.4.1_提取第一次入院出现骨髓抑制.xlsx')
df_MTX_bmd_test_first.to_excel(writer)
writer.save()

In [329]:
# 统计第一次入院前就骨髓抑制
df_MTX_bmd_before=df_MTX_bmd_test_first[df_MTX_bmd_test_first['bmd_before']==1]

In [330]:
print(df_MTX_bmd_before.shape)
print(df_MTX_bmd_before['patient_id'].nunique())

(178, 44)
178


In [331]:
# 统计第一次用药3天后骨髓抑制
df_MTX_bmd_3d=df_MTX_bmd_test_first[(df_MTX_bmd_test_first['bmd_3d']==1)&
                                                             (df_MTX_bmd_test_first['bmd_before']==0)]

In [332]:
print(df_MTX_bmd_3d.shape)
print(df_MTX_bmd_3d['patient_id'].nunique())

(44, 44)
44


In [333]:
# 统计第一次用药7天后骨髓抑制
df_MTX_bmd_7d=df_MTX_bmd_test_first[(df_MTX_bmd_test_first['bmd_7d']==1)&
                                                              (df_MTX_bmd_test_first['bmd_3d']==0)&
                                                             (df_MTX_bmd_test_first['bmd_before']==0)]

In [334]:
print(df_MTX_bmd_7d.shape)
print(df_MTX_bmd_7d['patient_id'].nunique())

(9, 44)
9


In [335]:
# 统计第一次用药14天后骨髓抑制
df_MTX_bmd_14d=df_MTX_bmd_test_first[(df_MTX_bmd_test_first['bmd_14d']==1)&
                                                               (df_MTX_bmd_test_first['bmd_7d']==0)&
                                                              (df_MTX_bmd_test_first['bmd_3d']==0)&
                                                             (df_MTX_bmd_test_first['bmd_before']==0)]

In [336]:
print(df_MTX_bmd_14d.shape)
print(df_MTX_bmd_14d['patient_id'].nunique())

(1, 44)
1


In [337]:
# 统计第一次用药后没有出现骨髓抑制
df_MTX_bmd_no=df_MTX_bmd_test_first[(df_MTX_bmd_test_first['bmd_14d']==0)&
                                                               (df_MTX_bmd_test_first['bmd_7d']==0)&
                                                              (df_MTX_bmd_test_first['bmd_3d']==0)&
                                                             (df_MTX_bmd_test_first['bmd_before']==0)]

In [338]:
print(df_MTX_bmd_no.shape)
print(df_MTX_bmd_no['patient_id'].nunique())

(486, 44)
486


### 提取第一次发生骨髓抑制

In [339]:
# 取7天内天作为不良反应发生时间提取范围
df_MTX_bmd=pd.concat([df_MTX_bmd_3d,df_MTX_bmd_7d],axis=0)
df_MTX_bmd=df_MTX_bmd.reset_index(drop=True)

In [340]:
print(df_MTX_bmd.shape)
print(df_MTX_bmd['patient_id'].nunique())

(53, 44)
53


In [341]:
df_MTX_bmd['age'].describe()

count    53.000000
mean     22.547170
std      12.053672
min       6.000000
25%      14.000000
50%      18.000000
75%      32.000000
max      48.000000
Name: age, dtype: float64

In [342]:
# 保存第一次入院发生不良反应
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.4.2_提取MTX用药后第一次出现骨髓抑制.xlsx')
df_MTX_bmd.to_excel(writer)
writer.save()

### 提取第一次发生骨髓抑制时间

In [343]:
# 计算第一次用药到第一次发生不良反应时间间隔=test_date - start_datetime
temp_list=[]
for i in df_MTX_bmd['case_no'].unique():
    # print(i)
    temp=df_MTX_bmd[df_MTX_bmd['case_no']==i]
    temp=temp.reset_index(drop=True)
    temp_bmd=df_test_bmd_low[(df_test_bmd_low['case_no']==i) &
        (df_test_bmd_low['test_date']>=temp.loc[0,'start_datetime']+datetime.timedelta(days=1))]
    temp_bmd=temp_bmd.sort_values(['test_date'])
    temp_bmd=temp_bmd.reset_index(drop=True)
    for j in range(1,temp_bmd.shape[0]):
        # 如果出现2个降低的不同指标，则认为发生骨髓抑制，计算发生时间
        if temp_bmd.loc[0,'project_name'] != temp_bmd.loc[j,'project_name']:
            t = (temp_bmd.loc[j,'test_date']-temp.loc[0,'start_datetime']).days
            # 发生不良反应的时间
            temp.loc[0,'bmd_time']=temp_bmd.loc[j,'test_date']
            # 服药到发生不良反应的时间间隔
            temp.loc[0,'start_bmd_time']=t
            break
    temp_list.append(temp)

In [344]:
df_MTX_bmd_time=temp_list[0]
for j in range(1,len(temp_list)):
    df_MTX_bmd_time=pd.concat([df_MTX_bmd_time,temp_list[j]],axis=0)
df_MTX_bmd_time=df_MTX_bmd_time.reset_index(drop=True)
del temp_list

In [345]:
df_MTX_bmd_time=df_MTX_bmd_time.sort_values(['case_no','start_datetime'])
df_MTX_bmd_time=df_MTX_bmd_time.reset_index(drop=True)

In [346]:
# 保存第一次入院第一次发生不良反应时间间隔
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.4.3_计算MTX用药-发生骨髓抑制时间间隔.xlsx')
df_MTX_bmd_time.to_excel(writer)
writer.save()

## 骨髓抑制打标签

In [347]:
# 用药7天内新出现骨髓抑制算发生不良反应-1，其他的情况为0
df_MTX_bmd_time['bmd_label']=1

In [348]:
df_MTX_bmd_0=pd.concat([df_MTX_bmd_before,df_MTX_bmd_14d],axis=0)
df_MTX_bmd_0=pd.concat([df_MTX_bmd_0,df_MTX_bmd_no],axis=0)
df_MTX_bmd_0['bmd_label']=0

In [349]:
# 将无不良反应的bmd_time定义为服药后第七天
df_MTX_bmd_0['bmd_time']=df_MTX_bmd_0['start_datetime'].apply(lambda x: x+datetime.timedelta(days=7))

In [350]:
df_MTX_bmd_label=pd.concat([df_MTX_bmd_time,df_MTX_bmd_0],axis=0)
df_MTX_bmd_label=df_MTX_bmd_label.reset_index(drop=True)

In [351]:
print(df_MTX_bmd_label.shape)
print(df_MTX_bmd_label['patient_id'].nunique())

(718, 47)
718


In [352]:
# 保存甲氨蝶呤骨髓抑制分组
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_5.5_甲氨蝶呤骨髓抑制打标签.xlsx')
df_MTX_bmd_label.to_excel(writer)
writer.save()

# 添加甲氨蝶呤TDM检测

In [245]:
# 提取用药后不良反应发生前的甲氨蝶呤血药浓度检测
df_test_MTX=df_test_result[df_test_result['project_name'].str.contains('甲氨蝶呤|MTX')]
df_test_MTX=df_test_MTX.reset_index(drop=True)

print(df_test_MTX.shape)
print(df_test_MTX['patient_id'].nunique())

(13483, 11)
1437


In [246]:
# 提取患者的tdm检测
df_test_MTX=df_test_MTX[df_test_MTX['case_no'].isin(df_MTX_bmd_label['case_no'].unique())]
df_test_MTX=df_test_MTX.sort_values(['case_no','test_date'])
df_test_MTX=df_test_MTX.reset_index(drop=True)

In [513]:
print(df_test_MTX.shape)
print(df_test_MTX['patient_id'].nunique())

(3396, 11)
709


In [276]:
# 保存甲氨蝶呤血药浓度检测
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_temp_甲氨蝶呤血药浓度检测数据.xlsx')
df_test_MTX.to_excel(writer)
writer.save()

In [273]:
# 能明确的是，甲氨蝶呤case_no对应的tdm检测主要是"MTX输药后"
# tdm_24,tdm_48可能时间在同一天，这很正常。
# tdm主要取24、48、72等检测时间判别不良反应，0h过1000表明有效性，如果不能在服药24h后低于0.1，则认为发生不良反应，需要持续观测
# 就算24h后发生不良反应，血液提取的tdm72也没问题，保留

df_test_MTX_select=df_test_MTX[df_test_MTX['project_name'].str.contains('输药后')]
df_test_MTX_select=df_test_MTX_select.drop_duplicates(['case_no','project_name'],keep='first')
df_test_MTX_select=df_test_MTX_select.sort_values(['case_no','test_date','test_result'])

In [274]:
df_test_MTX_select.shape

(3193, 11)

In [380]:
# 提取输药后24小时TDM检测浓度
df_test_MTX_24=df_test_MTX_select[df_test_MTX_select['project_name'].str.contains('MTX输药后24H')]
df_test_MTX_24=df_test_MTX_24[['case_no','test_date','test_result']]
df_test_MTX_24=df_test_MTX_24.reset_index(drop=True)

# 提取输药后48小时TDM检测浓度
df_test_MTX_48=df_test_MTX_select[df_test_MTX_select['project_name'].str.contains('MTX输药后48H')]
df_test_MTX_48=df_test_MTX_48[['case_no','test_date','test_result']]
df_test_MTX_48=df_test_MTX_48.reset_index(drop=True)

# 提取输药后72小时TDM检测浓度
df_test_MTX_72=df_test_MTX_select[df_test_MTX_select['project_name'].str.contains('MTX输药后72H')]
df_test_MTX_72=df_test_MTX_72[['case_no','test_date','test_result']]
df_test_MTX_72=df_test_MTX_72.reset_index(drop=True)

In [386]:
df_test_MTX_72

,case_no,test_date,test_result
0,1000527,2017-05-16,3.44
1,1004623,2017-05-29,1.34
2,1005304,2017-05-31,0.1
3,1005362,2017-06-02,7.64
4,1006139,2017-06-05,0.1
...,...,...,...
589,993527,2017-04-24,0.1
590,997106,2017-05-05,0.1
591,997291,2017-05-05,0.63
592,998241,2017-05-09,0.1


In [389]:
# 合并甲氨蝶呤tdm检测信息

temp_list=[]
for i in np.unique(df_MTX_bmd_label['case_no']):
    temp=df_MTX_bmd_label[df_MTX_bmd_label['case_no']==i]
    temp=temp.reset_index(drop=True)
    # 如果存在输药后24小时
    if i in np.unique(df_test_MTX_24['case_no']):
        temp_MTX_24=df_test_MTX_24[df_test_MTX_24['case_no']==i]
        temp_MTX_24=temp_MTX_24.reset_index(drop=True)
        temp['MTX_tdm_24h']=temp_MTX_24.loc[0,'test_result']
    # 如果存在输药后48小时
    if i in np.unique(df_test_MTX_48['case_no']):
        temp_MTX_48=df_test_MTX_48[df_test_MTX_48['case_no']==i]
        temp_MTX_48=temp_MTX_48.reset_index(drop=True)
        temp['MTX_tdm_48h']=temp_MTX_48.loc[0,'test_result']
    # 如果存在输药后72小时
    if i in np.unique(df_test_MTX_72['case_no']):
        temp_MTX_72=df_test_MTX_72[df_test_MTX_72['case_no']==i]
        temp_MTX_72=temp_MTX_72.reset_index(drop=True)
        temp['MTX_tdm_72h']=temp_MTX_72.loc[0,'test_result']
    temp_list.append(temp)

In [390]:
df_MTX_tdm=temp_list[0]
for j in range(1,len(temp_list)):
    df_MTX_tdm=pd.concat([df_MTX_tdm,temp_list[j]],axis=0)
df_MTX_tdm=df_MTX_tdm.reset_index(drop=True)
del temp_list

print(df_MTX_tdm.shape)
print(df_MTX_tdm['patient_id'].nunique())
print(df_MTX_tdm['case_no'].nunique())

(718, 50)
718
718


In [391]:
# 保存添加tdm检测的甲氨蝶呤用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_6_甲氨蝶呤添加tdm检测.xlsx')
df_MTX_tdm.to_excel(writer)
writer.save()

# 合并个人史、既往史、生理病理

In [399]:
# 个人史包括吸烟史、饮酒史
# 既往史包括高血压、糖尿病、青霉素过敏史、肝炎、结核史、手术外伤史、输血史、预防接种史
df_history=pd.DataFrame(columns=['case_no','吸烟史','饮酒史','高血压','糖尿病','肝炎'])
df_history['case_no']=df_medical_record['case_no'].unique()
# dataframe空值填充
df_history.fillna(0,inplace=True)
# 将patient_id设为索引
df_history=df_history.set_index('case_no')

In [516]:
df_history.shape

(34406, 6)

In [400]:
temp_list=[]
for i in range(df_medical_record.shape[0]):
    case_no=df_medical_record.loc[i,'case_no']
    # 判断个人史、既往史等
    record_content=df_medical_record.loc[i,'record_content']
    record_type=df_medical_record.loc[i,'record_type']
    record_list=re.split('，|。',record_content)
    # 匹配个人史
    if record_type == '个人史':
        for j in record_list:
            # 匹配吸烟史
            if re.match('[^无否戒]*烟',j):
                if case_no == '1178717':
                    print(j)
                df_history.loc[case_no,'吸烟史']=1
            # 匹配饮酒史
            if re.match('[^无否戒]*酒[^精]*',j):
                df_history.loc[case_no,'饮酒史']=1
    # 匹配既往史
    if record_type == '既往史':
        for j in record_list:
            # 匹配高血压
            if re.match('[^无否]*高血压[^不详]*',j):
                df_history.loc[case_no,'高血压']=1
            # 匹配糖尿病
            if re.match('[^无否]*糖尿病',j):
                df_history.loc[case_no,'糖尿病']=1
            # 匹配肝炎
            if re.match('[^无否]*肝炎',j):
                df_history.loc[case_no,'肝炎']=1

In [401]:
# 取消patient_id索引
df_history=df_history.reset_index()

In [402]:
df_history

,case_no,吸烟史,饮酒史,高血压,糖尿病,肝炎
0,352356,0,0,0,0,0
1,355679,0,0,0,0,0
2,355836,0,0,0,0,0
3,357506,0,0,0,0,0
4,362433,0,0,0,0,0
...,...,...,...,...,...,...
34401,1476825,0,0,0,0,0
34402,1476486,0,0,0,0,0
34403,1476302,0,0,0,0,0
34404,1476706,0,0,0,0,0


In [403]:
# 保存个人史既往史
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_temp_保存个人史既往史.xlsx')
df_history.to_excel(writer)
writer.save()

In [455]:
# 合并个人史既往史
df_MTX_history=pd.merge(df_MTX_tdm,df_history,on=['case_no'],how='left')

In [456]:
print(df_MTX_history.shape)
print(df_MTX_history['patient_id'].nunique())

(718, 55)
718


In [457]:
# 保存联合用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_7_合并个人史既往史.xlsx')
df_MTX_history.to_excel(writer)
writer.save()

# 增加联合用药

In [459]:
# 提取患者的联合用药
df_drug_other=df_doctor_order[~ df_doctor_order['drug_name'].str.contains('甲氨蝶呤|甲氨喋呤|MTX|葡萄糖|氯化钠')]
# 患者限定
df_drug_other=df_drug_other[df_drug_other['case_no'].isin(df_MTX_history['case_no'].unique())]
df_drug_other=df_drug_other.reset_index(drop=True)

In [460]:
# 联合用药时间限定：用药前3天内-不良反应发生前
temp_list=[]
for i in df_drug_other['case_no'].unique():
    temp=df_drug_other[df_drug_other['case_no']==i]
    temp=temp.reset_index(drop=True)
    # MTX用药时间
    temp_drug=df_MTX_history[df_MTX_history['case_no']==i]
    temp_drug=temp_drug.reset_index(drop=True)
    drug_time=temp_drug.loc[0,'start_datetime']
    bmd_time=temp_drug.loc[0,'bmd_time']
    # 提取用药前3天内-不良反应发生前的联合用药。各自反集取交集
    temp=temp[(temp['end_datetime']<=drug_time-datetime.timedelta(days=3)) &
             (temp['start_datetime']>=bmd_time)]
    temp=temp.sort_values(['start_datetime'])
    temp_list.append(temp)

In [461]:
df_drug_other=temp_list[0]
for j in range(1,len(temp_list)):
    df_drug_other=pd.concat([df_drug_other,temp_list[j]],axis=0)
df_drug_other=df_drug_other.reset_index(drop=True)
del temp_list

In [518]:
df_drug_other.shape

(5917, 11)

In [462]:
df_drug_other_name=pd.DataFrame(df_drug_other.drug_name.value_counts())
# 保存联合用药名
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_temp_联合用药名.xlsx')
df_drug_other_name.to_excel(writer)
writer.save()

In [463]:
# 主药：多柔比星、多柔比星脂质体、顺铂、异环磷酰胺、依托泊苷
df_ADM = df_drug_other[df_drug_other['drug_name'].str.contains('注射用盐酸多柔比星')]
df_IFO = df_drug_other[df_drug_other['drug_name'].str.contains('异环磷酰胺|IFO')]
df_CDP = df_drug_other[df_drug_other['drug_name'].str.contains('顺铂|CDP')]
df_VP = df_drug_other[df_drug_other['drug_name'].str.contains('依托泊苷|VP')]
df_drug_zhi = df_drug_other[df_drug_other['drug_name'].str.contains('多柔比星脂质体')]

# 辅助用药：头孢类、促红素、抗凝血、吡柔比星、保肝药、胃癌药
df_drug_toubao =  df_drug_other[df_drug_other['drug_name'].str.contains('头孢')]
df_drug_EPO = df_drug_other[df_drug_other['drug_name'].str.contains('促红素')]
df_drug_THP = df_drug_other[df_drug_other['drug_name'].str.contains('吡柔比星|吡柔吡星|THP')]
df_drug_kang = df_drug_other[df_drug_other['drug_name'].str.contains('美司钠|肝素钠')]
df_drug_GSH = df_drug_other[df_drug_other['drug_name'].str.contains('谷胱甘肽')]
df_drug_ganmei = df_drug_other[df_drug_other['drug_name'].str.contains('异甘草酸镁')]
df_drug_PO4Na = df_drug_other[df_drug_other['drug_name'].str.contains('地塞米松磷酸钠')]
df_drug_Tro = df_drug_other[df_drug_other['drug_name'].str.contains('托烷司琼')]
df_drug_gangan = df_drug_other[df_drug_other['drug_name'].str.contains('甘草酸苷')]
df_drug_VCR = df_drug_other[df_drug_other['drug_name'].str.contains('长春新碱')]
df_drug_ppt = df_drug_other[df_drug_other['drug_name'].str.contains('多烯磷脂酰')]

In [464]:
# 合并联合用药
df_MTX_drug_other=df_MTX_history.copy()
df_MTX_drug_other['多柔比星']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_ADM['case_no'].unique() else 0)
df_MTX_drug_other['异环磷酰胺']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_IFO['case_no'].unique() else 0)
df_MTX_drug_other['顺铂']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_CDP['case_no'].unique() else 0)
df_MTX_drug_other['依托泊苷']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_VP['case_no'].unique() else 0)
df_MTX_drug_other['多柔比星脂质体']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_zhi['case_no'].unique() else 0)

df_MTX_drug_other['头孢类']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_toubao['case_no'].unique() else 0)
df_MTX_drug_other['促红素']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_EPO['case_no'].unique() else 0)
df_MTX_drug_other['吡柔吡星']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_THP['case_no'].unique() else 0)
df_MTX_drug_other['抗凝血']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_kang['case_no'].unique() else 0)

df_MTX_drug_other['谷胱甘肽']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_GSH['case_no'].unique() else 0)
df_MTX_drug_other['异甘草酸镁']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_ganmei['case_no'].unique() else 0)
df_MTX_drug_other['地塞米松磷酸钠']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_PO4Na['case_no'].unique() else 0)
df_MTX_drug_other['托烷司琼']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_Tro['case_no'].unique() else 0)
df_MTX_drug_other['甘草酸苷']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_gangan['case_no'].unique() else 0)
df_MTX_drug_other['长春新碱']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_VCR['case_no'].unique() else 0)
df_MTX_drug_other['多烯磷脂酰']=df_MTX_drug_other['case_no'].astype('str').apply(lambda x: 1 if x in df_drug_ppt['case_no'].unique() else 0)

In [465]:
# 保存联合用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_8_合并联合用药.xlsx')
df_MTX_drug_other.to_excel(writer)
writer.save()

# 添加治疗方案

In [ ]:
# 根据院方提供的治疗方案，进行标注
# 单药：甲氨蝶呤、多柔比星、多柔比星脂质体、顺铂、异环磷酰胺、依托泊苷
# 2种联合方案：甲氨蝶呤+多柔比星，甲氨蝶呤+多柔比星脂质体，甲氨蝶呤+顺铂，甲氨蝶呤+异环磷酰胺，甲氨蝶呤+依托泊苷
# 3种联合方案：甲氨蝶呤+多柔比星+顺铂，甲氨蝶呤+多柔比星+异环磷酰胺，甲氨蝶呤+多柔比星+依托泊苷；
            #甲氨蝶呤+多柔比星脂质体+顺铂，甲氨蝶呤+多柔比星脂质体+异环磷酰胺，甲氨蝶呤+多柔比星脂质体+依托泊苷；
            #甲氨蝶呤+顺铂+异环磷酰胺，甲氨蝶呤+顺铂+依托泊苷
            #甲氨蝶呤+异环磷酰胺+依托泊苷
# 4种方案：甲氨蝶呤+多柔比星+顺铂+异环磷酰胺、甲氨蝶呤+多柔比星+顺铂+依托泊苷、甲氨蝶呤+多柔比星+异环磷酰胺+依托泊苷；
        #甲氨蝶呤+多柔比星脂质体+顺铂+异环磷酰胺、甲氨蝶呤+多柔比星脂质体+顺铂+依托泊苷、甲氨蝶呤+多柔比星脂质体+异环磷酰胺+依托泊苷；
        #甲氨蝶呤+顺铂+异环磷酰胺+依托泊苷
# 5种方案：甲氨蝶呤+多柔比星+顺铂+异环磷酰胺+依托泊苷；甲氨蝶呤+多柔比星脂质体+顺铂+异环磷酰胺+依托泊苷

In [466]:
# 判断一个列表中是否同时存在两个元素，存在返回True，否则False
def judge_list_element(list1,e1,e2):
    if e1 in list1 and e2 in list1:
        return True
    else:
        return False

In [467]:
from itertools import combinations, permutations
drug_case=['多柔比星','多柔比星脂质体','顺铂','异环磷酰胺','依托泊苷']
df_MTX_treatcase=df_MTX_drug_other.copy()
df_MTX_treatcase=df_MTX_treatcase.reset_index(drop=True)
for i in range(df_MTX_treatcase.shape[0]):
    for j in range(1,len(drug_case)):
        # 计算排列组合数
        drug_case_set=combinations(drug_case,j)
        for k in drug_case_set:
            list1=list(k)
            if judge_list_element(list1,'多柔比星','多柔比星脂质体'):
                continue
            if (df_MTX_treatcase.loc[i,list1]).sum() == len(list1):
                list1.insert(0,'甲氨蝶呤')
                df_MTX_treatcase.loc[i,'+'.join(list1)] = 1
            else:
                list1.insert(0,'甲氨蝶呤')
                df_MTX_treatcase.loc[i,'+'.join(list1)] = 0

In [468]:
# 增加治疗方案
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_9_增加治疗方案.xlsx')
df_MTX_treatcase.to_excel(writer)
writer.save()

# 高低剂量组

In [469]:
df_MTX_group=df_MTX_treatcase.copy()

In [470]:
# 按15mg划分高低剂量组
df_MTX_group['group'] = df_MTX_group['日剂量'].astype('float').apply(lambda x: 1 if x>15 else 0)

In [471]:
print(df_MTX_group.shape)

(718, 95)


In [472]:
# 划分高低剂量组
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_10_划分高低剂量组.xlsx')
df_MTX_group.to_excel(writer)
writer.save()

# 合并其他相关检测

## 提取其他检测

In [473]:
# 提取甲氨蝶呤患者的所有检测
df_test_other=df_test_result[df_test_result['case_no'].isin(df_MTX_group['case_no'].unique())]

In [474]:
# 提取患者的其他检测
df_test_other = df_test_other[~ df_test_other['project_name'].str.contains('甲氨蝶呤|MTX|血红蛋白|白细胞|粒细胞|血小板|尿|粪|大便')]
df_test_other = df_test_other[~ df_test_other['project_code'].str.contains('IG#|PLT')]
df_test_other = df_test_other.reset_index(drop=True)

In [521]:
df_test_other.to_csv('temp_test_other.csv')

In [519]:
df_test_other.shape

(138315, 11)

In [476]:
df_test_other['project_name'].value_counts()

红细胞          5354
PH           2687
颜色           2684
细胞比积         2679
RBC平均容量      2672
             ... 
拒收原因            1
流感病毒B型IgM       1
内毒素试验           1
CH50            1
肺炎衣原体IgM        1
Name: project_name, Length: 262, dtype: int64

In [477]:
print(df_test_other['project_name'].nunique())

262


In [478]:
df_test_other_name=pd.DataFrame(df_test_other['project_name'].value_counts())
# 划分高低剂量组
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_temp_其他检测项.xlsx')
df_test_other_name.to_excel(writer)
writer.save()

## 合并其他检测

In [494]:
def judge_float(x):
    try:
        a=float(x)
        return a
    except:
        return np.NaN

In [503]:
# 将用药后不良反应发生前的最近一次其他检测转置到一行中
for i in range(df_MTX_group.shape[0]):
    case_no=df_MTX_group.loc[i,'case_no']
    temp=df_MTX_group[df_MTX_group['case_no']==case_no]
    temp=temp.reset_index(drop=True)
    
    # case_no的其他检测
    temp_other= df_test_other[df_test_other['case_no']==case_no]
    temp_other=temp_other.reset_index(drop=True)
    # 取用药后不良反应发生前的其他检测
    temp_other_1=temp_other[(temp_other['test_date']>=temp.loc[0,'start_datetime']+datetime.timedelta(days=1))&
                            (temp_other['test_date']<=temp.loc[0,'bmd_time'])]
    # 取最近的一次其他检测
    temp_other_1=temp_other_1.sort_values(['test_date'])
    temp_other_1=temp_other_1.drop_duplicates(['project_name'],keep='last')
    temp_other_1=temp_other_1.reset_index(drop=True)
    
    # 检验项转置为表头
    if temp_other_1.shape[0]>0:
        temp_other_1['test_result']=temp_other_1['test_result'].apply(judge_float)
        temp_other_pivot=temp_other_1.pivot_table('test_result',['case_no'], 'project_name')
        temp_other_pivot=temp_other_pivot.reset_index(drop=True)
        for j in temp_other_pivot.columns[1:]:
            df_MTX_group.loc[i,j]=temp_other_pivot.loc[0,j]
#     if case_no == '410308':
#         print(temp.loc[0,'start_datetime'])
#         print(temp.loc[0,'bmd_time'])
#         print(temp_other_1)
#         print(temp_other_2[temp_other_2['project_name']=='肌酐(干式)'])
#         break

In [504]:
print(df_MTX_group.shape)

(718, 225)


In [505]:
# 合并其他检测
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_11.2_合并其他检测.xlsx')
df_MTX_group.to_excel(writer)
writer.save()

## 删除缺失率超过50%的列

In [506]:
# 删除缺失超过50%的检测
for i in df_MTX_group.columns:
    x=df_MTX_group[i].isnull().sum()
    per_x = x/df_MTX_group.shape[0]
    if per_x >= 0.5:
        print(i)
        del df_MTX_group[i]

血红蛋白_7d
血红蛋白_14d
白细胞_7d
白细胞_14d
粒细胞_3d
粒细胞_7d
粒细胞_14d
血小板_7d
血小板_14d
血红蛋白_7d_B
血红蛋白_14d_B
白细胞_7d_B
白细胞_14d_B
粒细胞_3d_B
粒细胞_7d_B
粒细胞_14d_B
血小板_7d_B
血小板_14d_B
start_bmd_time
上皮细胞
二氧化碳(干式)
导电率信息
小圆上皮细胞
病理管型
管型
精子
PH
凝血酶原时间
凝血酶时间
国际标准化比率
抗凝血酶Ⅲ活性
纤维蛋白(原）降解产物
纤维蛋白原
部分凝血活酶时间
无定形结晶
有核红细胞比例
有核红细胞绝对值
草酸钙结晶
移形上皮细胞
快速C反应蛋白
.  CD3
.  CD4
.  CD4/CD8
.  CD8
.  NK
CI(凝血功能综合指数)
EPL(纤维蛋白溶解状态)
K(纤维蛋白原功能)
LY30(纤维蛋白溶解状态)
R(凝血因子活性)
宽大管型
总25-羟基维生素D
骨钙素N端中分子(N-MID)
抗丙肝病毒抗体
内毒素试验
真菌G试验
降钙素原
单核细胞%
涂抹细胞%
淋巴细胞%
灰尘%
γ-谷氨酰酶
二氧化碳
前白蛋白
总胆汁酸
总胆红素
总蛋白
白/球比例
白蛋白
直接胆红素
碱性磷酸酶
磷
类酵母菌
肌酐
胆碱酯酶
血清氯
血清钠
血清钾
视黄醇结合蛋白
谷丙转氨酶
谷草转氨酶
钙
镁
α1球蛋白
α2球蛋白
β球蛋白
γ球蛋白
清蛋白
a/A
二氧化碳分压
二氧化碳总量
全血碱剩余
标准重碳酸盐
氧分压
氧饱和度
碳酸氢根浓度
细胞外液剩余碱
肺泡内氧分压
钠
钾
异型淋巴细胞
血沉
叶酸
总铁结合力
维B12
网织红细胞比值
网织红细胞绝对值
血清铁
铁蛋白(FERRITIN)
线粒体谷草转氨酶
胱抑素-C
血糖(干式)
D二聚体


In [507]:
print(df_MTX_group.shape)

(718, 112)


In [508]:
    # 保存删除缺失率后的数据
    writer=pd.ExcelWriter(project_path+'/data/processed_data/df_11.3_删除缺失超过50%的列.xlsx')
    df_MTX_group.to_excel(writer)
    writer.save()

## 人工删除无意义列

In [509]:
del df_MTX_group['未分类管型']
del df_MTX_group['比重']
del df_MTX_group['透明管型']
del df_MTX_group['粘液']
del df_MTX_group['未分类结晶']
del df_MTX_group['细胞比积']
del df_MTX_group['钙(干式)']
del df_MTX_group['非鳞状上皮细胞']
del df_MTX_group['鳞状上皮细胞']
# 离子类
del df_MTX_group['血清氯(干式)']
del df_MTX_group['血清钠(干式)']
del df_MTX_group['血清钾(干式)']

In [510]:
print(df_MTX_group.shape)

(718, 103)


In [511]:
# 保存删除缺失率后的数据
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_11.4_人工删除无意义列.xlsx')
df_MTX_group.to_excel(writer)
writer.save()